In [1]:
Packages <- c("dplyr","nleqslv", "broom","cubature", "geosphere", "data.table",  "ggplot2", "bbmle", "dplyr","tidyr", "stringr", "tidyverse", "lubridate", "RColorBrewer")

invisible(suppressPackageStartupMessages(lapply(Packages, library, character.only = TRUE)))

setwd('/local/home/katrinac/oceanography')
"%!in%" <- function(x,table) match(x,table, nomatch = 0) == 0
source("~/parentage/kernel_fitting/1340_loci/functions/ll_kt_both_bbmle.R")
source("~/parentage/kernel_fitting/1340_loci/functions/GenGausKernInt_sum0.5.R") #integrate_kernel_sum1
source("~/parentage/kernel_fitting/1340_loci/functions/GenGausKernInt_sum1.R") #integrate_kernel_sum0.5
source("~/parentage/kernel_fitting/1340_loci/functions/cdf_solve.R") #median
source("~/parentage/kernel_fitting/1340_loci/functions/cdf_solve90.R") #dist 90% retained

#clownfish metadata
load("~/parentage/r_data/total_sampling_across_years.RData")
load("~/parentage/r_data/sampled_area_each_year.RData")
#https://github.com/pinskylab/Clownfish_persistence/blob/master/Data/Script_outputs/cumulative_prop_hab_sampled_by_site.RData)
load("~/parentage/r_data/cumulative_prop_hab_sampled_by_site.RData")
#download.file(url = "https://github.com/pinskylab/genomics/blob/master/data/fish-obs.RData?raw=true", destfile = "~/parentage/r_data/fish-obs.RData")
fish_obs <- readRDS("~/parentage/r_data/fish-obs.RData") 
load("~/parentage/r_data/site_dist_info.RData")
#download.file(url = "https://github.com/pinskylab/Clownfish_persistence/blob/master/Data/Data_from_database/anem_db.RData?raw=true", destfile = "~/parentage/r_data/anem_db.RData")
load("~/parentage/r_data/anem_db.RData")
#download.file(url = "https://github.com/pinskylab/Clownfish_persistence/blob/master/Data/Data_from_database/dives_db.RData?raw=true", destfile = "~/parentage/r_data/dives_db.RData")
load("~/parentage/r_data/dives_db.RData")
#download.file(url = "https://github.com/pinskylab/Clownfish_persistence/blob/master/Data/Data_from_database/fish_db.RData?raw=true", destfile = "~/parentage/r_data/dives_db.RData")
load("~/parentage/r_data/fish_db.RData")
load("~/parentage/r_data/gps_db.RData")

#stop dplyr's courtesy warnings about grouping variables and summarise()
options(dplyr.summarise.inform=F) 

In [2]:
UnsampledSites <- c("SF1", "SF2", "SF3", "SF4", "SF5", "SF6", "Pangasugan", "CAI", "Other") 


In [2]:
#read in the df where each particle is an observation
Simulation2012 <- fread(file="~/oceanography/script_output/SimulationSummaryTables/SimConnLongForm2012YearSampled.csv", showProgress = T)
Simulation2013 <- fread(file="~/oceanography/script_output/SimulationSummaryTables/SimConnLongForm2013YearSampled.csv", showProgress = T)
Simulation2014 <- fread(file="~/oceanography/script_output/SimulationSummaryTables/SimConnLongForm2014YearSampled.csv", showProgress = T)

#this is about 3 GB so fread is fastest, but consider these options for dealing with a large csv
#https://inbo.github.io/tutorials/tutorials/r_large_data_files_handling/

#head(Simulation2012 %>% group_by(date, source, destination) %>% summarise(Check=n()))
#CAI should be 201718, it is, all good

In [3]:
#read in the centroids adjusted for the simulation, so the Magbangons combined 
centroids <- read.csv(file="~/oceanography/script_output/SurveyData/SimulationCentroids.csv", header=T, stringsAsFactors = F)
#read in the distance matrix adjusted for the simulation
distances <- read.csv(file="~/oceanography/script_output/SurveyData/SimulationAllDistances.csv", header=T, stringsAsFactors = F)


#proportion sampled
#add how well we sampled sites- proportion of habitat sampled
load("~/parentage/r_data/cumulative_prop_hab_sampled_by_site.RData")
PropSamp <- cumulative_prop_hab_sampled_by_site %>%
    mutate(total_possible_sample_anems = ifelse(site=="Caridad Proper", 4, total_possible_sample_anems) ) %>%
    mutate(total_prop_hab_sampled_anems_tidied= ifelse(site=="Caridad Proper" & total_anems_sampled==4, 1, total_prop_hab_sampled_anems_tidied) ) %>%
    mutate(total_possible_sample_anems = ifelse(site=="Sitio Lonas", total_anems_sampled, total_possible_sample_anems) ) %>%
    mutate(total_prop_hab_sampled_anems_tidied= ifelse(site=="Sitio Lonas", 1, total_prop_hab_sampled_anems_tidied) )%>%
    mutate(total_prop_hab_sampled_anems_tidied= ifelse(is.nan(total_prop_hab_sampled_anems_tidied), 0, total_prop_hab_sampled_anems_tidied) ) %>%
    dplyr::select(site, time_frame, end_year, total_prop_hab_sampled_anems_tidied) 

PropSamp$site <- gsub(". ", ".", PropSamp$site, fixed=TRUE) #fix spaces in Magbangon names

S.Mag <- PropSamp %>%#make a table for all of the S. Mabangon years/prop sampled. Then join to the DF, make a column adding S.Magbangon values to all rows of prop sampled, but only sub that value in for PropSamp in N.Magbangon rows, then rename N.Magbangon as Magbangon
    filter(site=="S.Magbangon") %>%
    dplyr::select(site, end_year, total_prop_hab_sampled_anems_tidied) %>%
    rename(S.MagVal="total_prop_hab_sampled_anems_tidied", extra="site")

PropSamp <- left_join(PropSamp, S.Mag, by=c("end_year")) %>%
    mutate(S.MagSum=total_prop_hab_sampled_anems_tidied+S.MagVal) %>% #create col adding the S.Mag values to prop hab - keep in mind values could be greater than 1- if so change them to 1
    mutate(total_prop_hab_sampled_anems_tidied= ifelse(site=="N.Magbangon", S.MagSum, total_prop_hab_sampled_anems_tidied)) %>%#sub this value in for only N.Mag
    mutate(site=ifelse(site=="N.Magbangon", "Magbangon", site)) %>%#change N.Mag name to generic Mag
    filter(site !="S.Magbangon") %>%#eliminate the S.Mag rows, they are now repeats
    dplyr::select(-S.MagVal,-extra, -S.MagSum, -time_frame) %>%
    rename(PropSamp="total_prop_hab_sampled_anems_tidied")


#check if there are values>1, should be none
sum(which(PropSamp$total_prop_hab_sampled_anems_tidied >1)) #zero, that's good

[1] 0

In [4]:
#load data from the genetic sampling in each year
#read in the kernel fitting summary
kernels <- read.csv(file="~/parentage/kernel_fitting/1340_loci/final_results/tables/kernel_fitting_summary.csv", header=T, stringsAsFactors = F)

##read in the sites that we sampled each year
N_gen_par <- read.table(file="~/parentage/colony2/20190523_1340loci/input/all_parents_corrected.txt", header = TRUE, stringsAsFactors = F) %>%#not sure that I need the parents here
    mutate(fish_indiv=as.character(fish_indiv))
N_gen_offs <- read.table(file="~/parentage/colony2/20190523_1340loci/input/all_offspring_corrected.txt", header=T, stringsAsFactors = F) %>%
    mutate(fish_indiv=as.character(fish_indiv))



#gather the summary of total offspring sampled
#from Allison, putting all the meta data together (Constants_database_common_functions.R)
##### Match up other relevant info (site, date, fish_indiv, etc.) to fish in the clownfish table
# Pull out year and month into a separate column in dives_db
dives_db_processed <- dives_db %>%
  mutate(year = as.integer(substring(date,1,4))) %>%
  mutate(month = as.integer(substring(date,6,7))) %>%
  mutate(dive_date = date(date))

# Pull all APCL caught or otherwise in the clownfish table
allfish_fish <- fish_db %>%
  select(fish_table_id, anem_table_id, fish_spp, sample_id, anem_table_id, recap, tag_id, color, sex, size, fish_obs_time, fish_notes) %>%
  filter(fish_spp == 'APCL') %>%
  mutate(size = as.numeric(size))  # make the size numeric (rather than chr) so can do means and such

# and their corresponding anemones
allfish_anems <- anem_db %>%
  select(anem_table_id, dive_table_id, anem_obs, anem_id, old_anem_id, anem_notes) %>%
  filter(anem_table_id %in% allfish_fish$anem_table_id)

# and the corresponding dive info
allfish_dives <- dives_db_processed %>%
  select(dive_table_id, dive_type, date, year, month, site, gps, dive_notes) %>%
  filter(dive_table_id %in% allfish_anems$dive_table_id) 

# join together
allfish_caught <- left_join(allfish_fish, allfish_anems, by="anem_table_id")
allfish_caught <- left_join(allfish_caught, allfish_dives, by="dive_table_id")

# add in the gen_ids and fish_indiv (now in a separate gen_id table) - gen_id only comes in the time the fish was sequenced, not at all captures
allfish_caught <- left_join(allfish_caught, (fish_obs %>% select(fish_table_id, gen_id, fish_indiv)), by = "fish_table_id") %>%
    select(fish_indiv, sample_id, site) %>%
    mutate()

N_gen_offs_annual  <- left_join(N_gen_offs, allfish_caught, by=c("fish_indiv", "sample_id")) %>% 
    group_by(year, site) %>%
    summarise(n_offs_gen=n()) %>%
    ungroup()

N_gen_offs_annual$site <- gsub(". ", ".", N_gen_offs_annual$site, fixed=TRUE)

##for all years
NGenOffsAll <- N_gen_offs_annual %>% 
    group_by(site) %>% 
    summarise(n_offs_gen=sum(n_offs_gen, na.rm=T))

sum(NGenOffsAll$n_offs_gen)

[1] 791

In [5]:
#combine N/S Magbangon in the genetic sampling data

AnnualRecsSamp <- bind_rows(N_gen_offs_annual %>%
                        filter(site !="N.Magbangon" & site!="S.Magbangon"),
                    N_gen_offs_annual %>%
                        mutate(Magbangon=ifelse(site=="N.Magbangon" | site=="S.Magbangon", "Magbangon", "no")) %>%
                        filter(Magbangon=="Magbangon") %>%
                        group_by(year, Magbangon) %>%
                        summarise(sum_offs=sum(n_offs_gen)) %>%
                        rename(site="Magbangon", n_offs_gen="sum_offs")) %>%
                        mutate(year=as.character(year)) %>%
                        ungroup()
sum(AnnualRecsSamp$n_offs_gen) #should be 791

#add in ALL sites that we sampled, even though we didn't find recruits there
AnnualRecsSamp <- left_join(PropSamp %>%  #add in all sampled sites as possible offs sites
                            filter(PropSamp > 0),
                          AnnualRecsSamp, 
                              by=c("site", end_year="year")) %>%
                        rename(year="end_year")  %>%
                        mutate(n_offs_gen=ifelse(is.na(n_offs_gen), 0, n_offs_gen)) %>%#if we didn't sample a recruit at a site that we still visited, put a 0 in place of the NA
                        filter(year %in% c("2012", "2013", "2014")) %>%  #* restrict to what we can pick up combining all years of simulation data
                        group_by(site) %>% #*
                        mutate(n_offs_gen_all_years=sum(n_offs_gen)) #*


[1] 791

In [87]:
for(i in 1:nrow(NGenOffs2014)){ 
    
    destination_eval <- as.character(NGenOffs2014$site[i]) #pick out a destination site
    
    SimSampDestination <- Simulation2014SurvGrowth %>% #USING DF WITH SURV/SIZE WEIGHTING ***
                                filter(destination==destination_eval) %>%
                                sample_n(as.numeric(NGenOffs2014$n_offs_gen[i]),  replace=F) %>% #sample particles that landed at that destination, number corresponding to the actual number sampled at that site in this year
                                select(one_of(names(Simulation2014)))#keep matching columns with the original df instead of carrying over the columns used for survival weighting
    SimulatedSampling2014 <- bind_rows(SimulatedSampling2014, SimSampDestination) #build into a sampled particle df
}


In [88]:
head(SimulatedSampling2014)

date,source,destination,SourceSampled,DestSampled,SimMonth,SimDay,SimYear,YearSampled,SimMonsoon,DailyParticles,ParticlesReleasedDaily,dist_km,bearing,direction
2014-01-14,SF3,Cabatoan,no,yes,1,1,2014,2014,NEM,42,2976,4.857907,-36.41887,323.5811
2013-12-22,Cabatoan,Cabatoan,yes,yes,12,12,2013,2014,NEM,14,992,0.000000,-180.00000,-10.0000
2013-12-26,Sitio Baybayon,Cabatoan,yes,yes,12,12,2013,2014,NEM,467,1984,24.910562,-12.35333,347.6467
2013-12-26,Sitio Baybayon,Cabatoan,yes,yes,12,12,2013,2014,NEM,467,1984,24.910562,-12.35333,347.6467
2014-01-07,Cabatoan,Cabatoan,yes,yes,1,1,2014,2014,NEM,68,992,0.000000,-180.00000,-10.0000
2014-01-21,SF2,Cabatoan,no,yes,1,1,2014,2014,NEM,117,3968,1.858614,-34.78244,325.2176


__Narrow down data to the year of interests__

In [44]:
#what if we account for post-settlement mortality and not sampling <3.5 cm recruits?
NumDays <- nrow(Simulation2012 %>% distinct(date))#200 days in a simulation

#make a df with weighted values for the days of the simluation oldest to most recent
SimDates <- Simulation2012 %>%
    distinct(date) %>%
    mutate(DateSeq=row_number()) %>% #assign row numbers, so more recent dates are higher than older
    mutate(SurvivalWeightExp=exp(DateSeq)) %>% #apply the exponential function to get a survival column
    mutate(SurvivalWeightExp=SurvivalWeightExp/max(SurvivalWeightExp))%>%#normalize as a survival "probability"
    mutate(SurvivalWeightLin=(((0.25/2)/5)*DateSeq)) %>%#maybe a linear survival function (y=mx)is more appropriate (eyeball from Allison's Fig 3C because what the hell why not we're just thinking here)
    mutate(SurvivalWeightLin=SurvivalWeightLin/max(SurvivalWeightLin)) #%>%#normalize as a survival "probability"
    #filter(DateSeq %!in% seq((max(DateSeq)-60), max(DateSeq), 1))#account for the fact that we didn't sample <3.5 cm
#the max survival prob will be different depending on when I filter out the more recent recruits. I don't want an artificially high prob of sampling based on a survival prob of 1 so I'm cutting fish out after prob weight calculations
Simulation2012SurvGrowth <- left_join(Simulation2012, SimDates, by="date") %>%
    filter(DateSeq %!in% seq((max(DateSeq)-60), max(DateSeq), 1))
nrow(Simulation2012SurvGrowth)<nrow(Simulation2012)#should be TRUE

[1] TRUE

In [45]:
#RUNNING ON 3/18/2021 ACCOUNTING FOR DIFFERENTIAL SAMPLING BASED ON SIZE/POSTSETTLEMENT SURVIVAL
#sample the simulation results according to how many offspring we sampled at that site in real life, each row is a column
NGenOffs2012 <- AnnualRecsSamp %>%
    filter(year=="2012")

nrow(NGenOffs2012) #should be 9 sites sampled for 2012

#make an empty dataframe with the same columns as original, this will be sampled particles
SimulatedSampling2012 = Simulation2012[FALSE,]

for(i in 1:nrow(NGenOffs2012)){ 
    
    destination_eval <- as.character(NGenOffs2012$site[i]) #pick out a destination site
    
    SimSampDestination <- Simulation2012 %>% #USING DF WITH SURV/SIZE WEIGHTING SurvGrowth ***
                                filter(destination==destination_eval) %>%
                                sample_n(as.numeric(NGenOffs2012$n_offs_gen[i]), replace=F) %>% #, weight=SurvivalWeightLin, sample particles that landed at that destination, number corresponding to the actual number sampled at that site in this year
                                select(one_of(names(Simulation2012)))#keep matching columns with the original df instead of carrying over the columns used for survival weighting
    SimulatedSampling2012 <- bind_rows(SimulatedSampling2012, SimSampDestination) #build into a sampled particle df
}

#assign a numeric ID for each row (which is a sampled particle)
SimulatedSampling2012 <- SimulatedSampling2012 %>%
    mutate(ParticleSampID=paste ("P", row_number(), sep = "", collapse = NULL))

nrow(SimulatedSampling2012)==sum(NGenOffs2012$n_offs_gen) #should be TRUE

#now randomly assign parentage match status to some of these rows
NumPar <- as.numeric(kernels %>%
    filter(Year=="2012") %>%
    select(NumParentageMatches))

SimulatedSampling2012Par <- SimulatedSampling2012 %>%
    ungroup() %>%
    filter(source %in% NGenOffs2012$site) %>% #we can only assign parentage if we sampled the source site, we've already ensured that we sampled the destination site in the for loop by indexing destination sites to evaluate 
    sample_n(NumPar, replace=F) %>% #sample number of rows consitent with numbers of parentage matches
    mutate(Parentage=1) #%>% #assign these rows a positive match value
    #arrange(date, source, destination) %>%#arrange by the same order as the orginal df, then bind columns together to avoid unexpected repeats from a join
    #select(date, source, destination, Parentage) #drop columns that will result in repeats 

#find the rows that haven't been assigned parentage
SimulatedSampling2012_2 <- anti_join((SimulatedSampling2012 %>% ungroup()), SimulatedSampling2012Par, by=c("ParticleSampID")) 

SimulatedSampling2012_2$Parentage <- 0 #add in the column for parentage

#combine parentage and non-parentage dfs
SimulatedSampling2012Par3 <- bind_rows(SimulatedSampling2012Par, SimulatedSampling2012_2) %>%
    mutate(YearSampled=as.character(YearSampled))

nrow(SimulatedSampling2012Par3) 
nrow(SimulatedSampling2012Par3 %>% filter(Parentage==1))

[1] 9

[1] TRUE

[1] 63

[1] 3

In [48]:
PropSamp2012 <- PropSamp %>%  #add in all sampled sites as possible parent sites
                filter(end_year =="2012" & PropSamp > 0) %>%
                #rename(source="site") %>%
                select(-end_year, -PropSamp)

#PropSamp2012$destination <- PropSamp2012$source #make another column for destination

#make a table with the total number of particles sampled at each site
TotalSimSamp <- SimulatedSampling2012Par3 %>% 
            select(source, destination) %>%
            group_by(destination) %>%
            summarise(NumSimSampRec=n()) %>%
            ungroup()
print("correct carry over of sampled particles in TotalSimSamp")
sum(TotalSimSamp$NumSimSampRec)==sum(NGenOffs2012$n_offs_gen) #should be true

#add in all surveyed sites
TotalSimSamp2 <- left_join(PropSamp2012, TotalSimSamp, by=c(site="destination")) %>% #join sites to destination, which will add a row for the surveyed sites with no samples
    mutate(NumSimSampRec=ifelse(is.na(NumSimSampRec), 0, NumSimSampRec)) %>% #change the Na's that result to 0s, because we didn't get samples there
    rename(destination="site")#rename sites to destination after joining

print("correct carry over of sampled particles in TotalSimSamp2")
sum(TotalSimSamp$NumSimSampRec)==sum(TotalSimSamp2$NumSimSampRec) #should be TRUE

#make a table with the total number of parentage matches found in the simulated loop
TotalSimPar <- SimulatedSampling2012Par3 %>% 
            select(source, destination, Parentage) %>%
            group_by(source,destination) %>%
            mutate(NumPar=sum(Parentage)) %>% #sum all the parentage observations along each ROUTE
            select(-Parentage) %>%
            ungroup() %>%
            distinct(source, destination, .keep_all = T)

print("correct number of parentage matches in TotalSimPar")
sum(TotalSimPar$NumPar)==NumPar #should be TRUE

#combine for a summary table BY DESTINATION, this below will be for UNASSIGNED ROWS
SimSampSummary2012 <- left_join(TotalSimSamp2, TotalSimPar, by="destination") %>%
    distinct(source, destination, .keep_all = T) %>%
    group_by(destination) %>%
    mutate(NumPar=ifelse(is.na(NumPar), 0, NumPar)) %>% #change the Na's that result to 0s, because we didn't get samples there
    mutate(NumPar=sum(NumPar)) %>% #sum all the parentage observations at each DESTINATION
    distinct(destination, .keep_all = T) %>%
    select(-source) %>%
    mutate(NumUnassigned=NumSimSampRec-NumPar)

print("correct number of destination sites, corresponding to number of surveyed sites in SimSampSummary2012, which is used for unassigned row")
nrow(SimSampSummary2012 %>% distinct(destination, .keep_all = T))==nrow(PropSamp2012) #should be TRUE, 9 possible destinations for 2012


#finally, join all of the sites surveyed for a complete parentage matrix
#for this, need to have a source/destination column for all sites
PropSamp2012 <- PropSamp2012 %>%
                rename(source="site")

PropSamp2012$destination <- PropSamp2012$source #make another column for destination

#this below will become parentage matrix! 
TotalSimPar2 <- full_join(TotalSimPar,PropSamp2012, by=c("source", "destination")) %>%
    mutate(NumPar=ifelse(is.na(NumPar), 0, NumPar))

print("correct number of destination sites, corresponding to number of surveyed sites in TotalSimPar2, which will be spread into parentage matrix")
nrow(TotalSimPar2 %>% distinct(destination, .keep_all = T))==nrow(PropSamp2012) #should be TRUE, 9 possible destinations for 2012

#check that the number of matches is still correct
print("correct number of parentage matches in TotalSimPar2, which will be spread into parentage matrix")
sum(TotalSimPar2$NumPar)==NumPar

#spread into matrix format
SimDispMat <- TotalSimPar2 %>%
    filter(source %in% NGenOffs2012$site) %>%
    arrange(source, destination)  %>%
    spread(destination, NumPar) %>%
    select(-source)
SimDispMat[is.na(SimDispMat)] <- 0

#add in unassigned row
unassigned <- t(as.matrix(SimSampSummary2012 %>% arrange(destination) %>% ungroup() %>% select(NumUnassigned)))
#rownames(unassigned)<-NULL
colnames(unassigned)<-names(SimDispMat)

SimDispMatFull2012 <- rbind(SimDispMat, unassigned)
#colnames(SimDispMatFull2012) <- NULL

#error check final matrix 
print("correct number of source sites in final matrix")
nrow(SimDispMatFull2012)==nrow(PropSamp2012)+1

print("correct number of destination sites in final matrix")
ncol(SimDispMatFull2012)==nrow(PropSamp2012)

print("correct number of parentage matches in final matrix")
sum(SimDispMatFull2012[1:nrow(PropSamp2012),])==NumPar
NumPar

print("correct number of unassigned recruits in final matrix")
sum(SimDispMatFull2012[nrow(PropSamp2012)+1,])==sum(NGenOffs2012$n_offs_gen)-NumPar
sum(NGenOffs2012$n_offs_gen)-NumPar

#finally- make sure dimensions of simulated parentage matrix match that of the genetic kernel fits. we should be totally replicating the genetic kernel fitting here
#EXCEPT! Remember that in the genetic data Magbangon was separated into two sites, but in the simulated data it's not
EmpiricalAssignments <- read.csv("~/parentage/kernel_fitting/1340_loci/input/parentage_matrix12_corrected.csv", header=F)

#print("dimensions of simulated and empirical parentage matrix are the same")
(dim(EmpiricalAssignments)-1)==dim(SimDispMatFull2012)

[1] "correct carry over of sampled particles in TotalSimSamp"


[1] TRUE

[1] "correct carry over of sampled particles in TotalSimSamp2"


[1] TRUE

[1] "correct number of parentage matches in TotalSimPar"


...

[1] 3

[1] "correct number of unassigned recruits in final matrix"


[1] TRUE

[1] 60

[1] TRUE TRUE

In [156]:
#everything should be true
Assignments <- SimDispMatFullAll
which(SimDispMat >0, arr.ind =T)==which(SimDispMatFullAll[1:nrow(PropSampAll),]>0, arr.ind =T)
#unassigned
which(SimDispMat >0, arr.ind =T)==which(Assignments[1:nrow(PropSampAll),]>0, arr.ind =T)
which(SimDispMatFullAll[nrow(PropSampAll)+1,]>0, arr.ind =T)==which(Assignments[nrow(PropSampAll)+1,]>0, arr.ind =T)

row,col
TRUE,TRUE
TRUE,TRUE
TRUE,TRUE
TRUE,TRUE
TRUE,TRUE
TRUE,TRUE
TRUE,TRUE
TRUE,TRUE
TRUE,TRUE
TRUE,TRUE


row,col
TRUE,TRUE
TRUE,TRUE
TRUE,TRUE
TRUE,TRUE
TRUE,TRUE
TRUE,TRUE
TRUE,TRUE
TRUE,TRUE
TRUE,TRUE
TRUE,TRUE


row,col
TRUE,TRUE
TRUE,TRUE
TRUE,TRUE
TRUE,TRUE
TRUE,TRUE
TRUE,TRUE
TRUE,TRUE
TRUE,TRUE
TRUE,TRUE
TRUE,TRUE


__Get all the components of the kernel fitting process together__

In [143]:
#assemble all of the components for annual kernel fitting
centroids <- read.csv("~/oceanography/empirical_data/site_centroids_simulation_kernels.csv", header=TRUE, stringsAsFactors = F)

Area <- read.csv("~/oceanography/empirical_data/site_area_header_nonsurveyed_simulation_kernels.csv", header=TRUE) %>%
    arrange(site) %>%
    filter(site %!in% c("near_north_full1", "near_north_full2", "near_north_full3", "near_south_full1", "near_south_full2", "near_south_full3")) %>%
    mutate(kmsq=msq*10^-6) #%>%
    #select(kmsq)
Reef_sizes <- as.matrix(Area)

#give every site in the distance matrix of the simulation (even if we didn't sample there) an index number
AllSites <- centroids %>%
    select(site) %>%
    arrange(site)
nrow(AllSites) #should be 34x1
AllSites$index <- seq(from=1, to=34, by=1)

SampledSites2012 <- inner_join(PropSamp2012 %>% select(source), AllSites, by=c(source="site")) 

#check for correct number of rows
nrow(SampledSites2012)==nrow(PropSamp2012) #should be true
SampledSites2012Index <- t(as.matrix(SampledSites2012$index))
ncol(SampledSites2012Index)

Sampled_reefs <- SampledSites2012Index

#proportion sampled matrix for kernel fitting

#for annual
Adult_sample_proportions <- PropSamp %>%  #add in all sampled sites as possible parent sites
                filter(end_year =="2012" & PropSamp > 0) 

Adult_sample_proportions <- as.matrix(Adult_sample_proportions$PropSamp)
nrow(Adult_sample_proportions) #should be 9 for 2012

#distance matrix using the centroids with combined Magbangon
### List of source locations
sites_source <- centroids

### List of destination locations
sites_dest <- centroids

dist_mat_m <- distm(sites_source[,c('lon','lat')], sites_source[,c('lon','lat')], fun=distVincentyEllipsoid)
Distances <- dist_mat_m*10^-3

Centroids <- centroids %>%
    select(-site)

Assignments <- SimDispMatFull2012


[1] 34

[1] TRUE

[1] 9

[1] 9

In [157]:
#do all of the site ids carry over properly
SampledSitesAll
PropSampAll
AllSites %>% filter(site %in% PropSampAll$source & site %!in% SampledSitesAll$source) #the site names are correct because this doesn't bring up a match
AllSites %>% filter(site %in% PropSampAll$source & index %!in% SampledSitesAll$index) #the site indexes are correct because this doesn't bring up a match


source,index
Cabatoan,1
Caridad Cemetery,8
Caridad Proper,9
Elementary School,14
Haina,16
Hicgop South,17
Magbangon,18
Palanas,25
Poroc Rose,26
Poroc San Flower,27


source
Cabatoan
Caridad Cemetery
Caridad Proper
Elementary School
Haina
Hicgop South
Magbangon
Palanas
Poroc Rose
Poroc San Flower


site,index


site,index


In [87]:
#check that the distance and area matrixes line up with the correct site ids
library(fields)
#centroids
Area <- read.csv("~/oceanography/empirical_data/site_area_header_nonsurveyed_simulation_kernels.csv", header=TRUE, stringsAsFactors = F) %>%
    arrange(site) %>%
    filter(site %!in% c("near_north_full1", "near_north_full2", "near_north_full3", "near_south_full1", "near_south_full2", "near_south_full3")) %>%
    mutate(kmsq=msq*10^-6)
not_matching <- which(Area$site!=centroids$site, arr.ind = T)
Area[not_matching,] #the near north/south rows won't match because their site ids are worded differently but that's fine

##calculate the distance from all potential parents and all potential offspring
all_possible_dists <- (distm(sites_source[,c('lon','lat')], sites_source[,c('lon','lat')], fun=distVincentyEllipsoid))
all_possible_dists <- as.data.frame(dist_mat_m*10^-3)

##attach the sample_ids to each distance, so you can also get site and year
colnames(all_possible_dists) <- centroids$site
all_possible_dists$site_i <- centroids$site

##gather into tidy df
all_possible_dists_tidy <- all_possible_dists %>%
    dplyr::select(site_i, everything()) %>%
    gather(2:35, key=site_j, value=dist_km) 
(all_possible_dists_tidy)



,site,msq,kmsq
19,near_north_ten_per_cover1,175000,0.175000
20,near_north_ten_per_cover2,175000,0.175000
21,near_north_ten_per_cover3,175000,0.175000
22,near_south_ten_per_cover1,158333,0.158333
23,near_south_ten_per_cover2,158333,0.158333
24,near_south_ten_per_cover3,158333,0.158333


site_i,site_j,dist_km
Cabatoan,Cabatoan,0.0000000
camotes_central1,Cabatoan,30.8887144
camotes_central2,Cabatoan,43.7121318
camotes_east1,Cabatoan,17.9721103
camotes_east2,Cabatoan,28.3601067
camotes_west1,Cabatoan,42.5550042
camotes_west2,Cabatoan,57.5651655
Caridad Cemetery,Cabatoan,3.0328732
Caridad Proper,Cabatoan,3.7812405
cuatras_islas_central,Cabatoan,39.5449097


In [88]:
which(t(all_possible_dists_tidy2$dist_km)-t(all_possible_dists_tidy$dist_km)>0.3, arr.ind = T)

row,col


In [140]:
as.matrix((all_possible_dists %>% select(Haina)))==as.matrix(Distances[,16]) #all true, so yes it looks like this code works fine
#does the index number match up?
AllSites %>% filter(index==16) #yes, all fine

Haina
TRUE
TRUE
TRUE
TRUE
TRUE
TRUE
TRUE
TRUE
TRUE
TRUE


site,index
Haina,16


In [7]:
#what is going on with 2014??
nrow(PropSamp %>% filter(end_year=="2014") %>% select(site))
nrow(PropSamp %>% filter(end_year=="2014")%>% filter(PropSamp > 0) %>% select(site))
nrow(N_gen_offs_annual %>% filter(year=="2014") %>% select(site))
nrow(AnnualRecsSamp %>% filter(year=="2014") %>% select(site))





[1] 18

[1] 16

[1] 11

[1] 16

In [8]:
#what are the sites in 2014 that we surveyed but didn't get recruits at?
SitesRec <- (N_gen_offs_annual %>% filter(year=="2014") %>% select(site))
SitesSurv <- PropSamp %>% filter(end_year=="2014")%>% filter(PropSamp > 0) %>% select(site)
SitesSurv %>% filter(site %!in% SitesRec$site)



site
Caridad Cemetery
Caridad Proper
Elementary School
Hicgop South
Magbangon
San Agustin
Sitio Lonas


In [9]:
nrow(PropSamp %>% filter(end_year=="2012") %>% select(site))
nrow(PropSamp %>% filter(end_year=="2012") %>% filter(PropSamp > 0) %>% select(site))
nrow(N_gen_offs_annual %>% filter(year=="2012") %>% select(site))
nrow(AnnualRecsSamp %>% filter(year=="2012") %>% select(site))




[1] 18

[1] 9

[1] 9

[1] 9

In [10]:
nrow(PropSamp %>% filter(end_year=="2013") %>% select(site))
nrow(PropSamp %>% filter(end_year=="2013")%>% filter(PropSamp > 0) %>% select(site) )
nrow(N_gen_offs_annual %>% filter(year=="2013") %>% select(site))
nrow(AnnualRecsSamp %>% filter(year=="2013") %>% select(site))

[1] 18

[1] 16

[1] 15

[1] 16

In [28]:

x <- list(Distances=Distances, Assignments=Assignments, Sampled_reefs=Sampled_reefs, Reef_sizes=Reef_sizes, Adult_sample_proportions=Adult_sample_proportions) #put inputs into a list because that's the bbmle format

Sim2012Fit <- suppressWarnings(mle2(LL_kt_bbmle, start=list(k=-2.51, theta=1.5), lower=c(-10, 0.15), upper=c(10, 8), method="L-BFGS-B", data=x, control=list(maxit=500)))
Sim2012Fit

BestK2012 <- as.numeric(coef(Sim2012Fit)[1])
BestTheta2012 <- as.numeric(coef(Sim2012Fit)[2])

SimMDD2012 <- cubintegrate(integrate_kernel_sum1, lower = 0, upper = Inf, k=BestK2012, theta=BestTheta2012, , method = "pcubature")$integral
SimMDD2012



Call:
mle2(minuslogl = LL_kt_bbmle, start = list(k = -2.51, theta = 1.5), 
    method = "L-BFGS-B", data = x, lower = c(-10, 0.15), upper = c(10, 
        8), control = list(maxit = 500))

Coefficients:
         k      theta 
-1.4347062  0.6571524 

Log-likelihood: -16.03 

[1] 9.855945

In [20]:
#awesome! it works! let's rip with a 100 iteration for loop
col <- c("year", "k", "theta", "mdd", "med", "dist90", "iteration")
SimulatedKernels2012 <- as.data.frame(matrix(nrow=0, ncol=7), stringsAsFactors = FALSE)
colnames(SimulatedKernels2012) <- col

#don't print warnings for this loop, they are only for setting row names in a tibble which is depracated. But it works. 
options(warn=-1)

pb <- txtProgressBar(min = 0, max = 100, style = 3)

StartTime <- Sys.time()

for(n in 1:100){
        
    #SAMPLE THE SIMULATION DATA, then fit a kernel
    
    #make an empty dataframe with the same columns as original, this will be sampled particles
    SimulatedSampling2012 = Simulation2012[FALSE,]
    
    for(i in 1:nrow(NGenOffs2012)){ 
        
        destination_eval <- as.character(NGenOffs2012$site[i]) #pick out a destination site
        
        SimSampDestination <- Simulation2012SurvGrowth %>% #USING DF WITH SURV/SIZE WEIGHTING ***SurvGrowth
                                filter(destination==destination_eval) %>%
                                sample_n(as.numeric(NGenOffs2012$n_offs_gen[i]),  replace=F) %>% #weight=SurvivalWeightLin,, sample particles that landed at that destination, number corresponding to the actual number sampled at that site in this year
                                select(one_of(names(Simulation2012)))#keep matching columns with the original df instead of carrying over the columns used for survival weighting
        
        SimulatedSampling2012 <- bind_rows(SimulatedSampling2012, SimSampDestination) #build into a sampled particle df
    }
    
    #assign a numeric ID for each row (which is a sampled particle)
    SimulatedSampling2012 <- SimulatedSampling2012 %>%
        mutate(ParticleSampID=paste ("P", row_number(), sep = "", collapse = NULL))
    
    #nrow(SimulatedSampling2012)==sum(NGenOffs2012$n_offs_gen) #should be TRUE
    
    #now randomly assign parentage match status to some of these rows
    NumPar <- as.numeric(kernels %>%
        filter(Year=="2012") %>%
        select(NumParentageMatches))
    
    SimulatedSampling2012Par <- SimulatedSampling2012 %>%
        ungroup() %>%
        filter(source %in% NGenOffs2012$site) %>% #we can only assign parentage if we sampled the source site, we've already ensured that we sampled the destination site in the for loop by indexing destination sites to evaluate 
        sample_n(NumPar, replace=F) %>% #sample number of rows consitent with numbers of parentage matches
        mutate(Parentage=1) #%>% #assign these rows a positive match value
        #arrange(date, source, destination) %>%#arrange by the same order as the orginal df, then bind columns together to avoid unexpected repeats from a join
        #select(date, source, destination, Parentage) #drop columns that will result in repeats 
    
    #find the rows that haven't been assigned parentage
    SimulatedSampling2012_2 <- anti_join((SimulatedSampling2012 %>% ungroup()), SimulatedSampling2012Par, by=c("ParticleSampID")) 
    
    SimulatedSampling2012_2$Parentage <- 0 #add in the column for parentage
    
    #combine parentage and non-parentage dfs
    SimulatedSampling2012Par3 <- bind_rows(SimulatedSampling2012Par, SimulatedSampling2012_2) %>%
        mutate(YearSampled=as.character(YearSampled))
    
    
    #NOW FORMAT INTO PARENTAGE MATRIX
    
                PropSamp2012 <- PropSamp %>%  #add in all sampled sites as possible parent sites
                            filter(end_year =="2012" & PropSamp > 0) %>%
                            #rename(source="site") %>%
                            select(-end_year, -PropSamp)

                #PropSamp2012$destination <- PropSamp2012$source #make another column for destination
                
                #make a table with the total number of particles sampled at each site
                TotalSimSamp <- SimulatedSampling2012Par3 %>% 
                            select(source, destination) %>%
                            group_by(destination) %>%
                            summarise(NumSimSampRec=n()) %>%
                            ungroup()
               
                
                #add in all surveyed sites
                TotalSimSamp2 <- left_join(PropSamp2012, TotalSimSamp, by=c(site="destination")) %>% #join sites to destination, which will add a row for the surveyed sites with no samples
                    mutate(NumSimSampRec=ifelse(is.na(NumSimSampRec), 0, NumSimSampRec)) %>% #change the Na's that result to 0s, because we didn't get samples there
                    rename(destination="site")#rename sites to destination after joining
                
                
                
                #make a table with the total number of parentage matches found in the simulated loop
                TotalSimPar <- SimulatedSampling2012Par3 %>% 
                            select(source, destination, Parentage) %>%
                            group_by(source,destination) %>%
                            mutate(NumPar=sum(Parentage)) %>% #sum all the parentage observations along each ROUTE
                            select(-Parentage) %>%
                            ungroup() %>%
                            distinct(source, destination, .keep_all = T)
                
                
                #combine for a summary table BY DESTINATION, this below will be for UNASSIGNED ROWS
                SimSampSummary2012 <- left_join(TotalSimSamp2, TotalSimPar, by="destination") %>%
                    distinct(source, destination, .keep_all = T) %>%
                    group_by(destination) %>%
                    mutate(NumPar=ifelse(is.na(NumPar), 0, NumPar)) %>% #change the Na's that result to 0s, because we didn't get samples there
                    mutate(NumPar=sum(NumPar)) %>% #sum all the parentage observations at each DESTINATION
                    distinct(destination, .keep_all = T) %>%
                    select(-source) %>%
                    mutate(NumUnassigned=NumSimSampRec-NumPar)
                                
                
                #finally, join all of the sites surveyed for a complete parentage matrix
                #for this, need to have a source/destination column for all sites
                PropSamp2012 <- PropSamp2012 %>%
                                rename(source="site")
                
                PropSamp2012$destination <- PropSamp2012$source #make another column for destination
                
                #this below will become parentage matrix! 
                TotalSimPar2 <- full_join(TotalSimPar,PropSamp2012, by=c("source", "destination")) %>%
                    mutate(NumPar=ifelse(is.na(NumPar), 0, NumPar))
        
                
                #spread into matrix format
                SimDispMat <- TotalSimPar2 %>%
                    filter(source %in% NGenOffs2012$site) %>%
                    arrange(source, destination)  %>%
                    spread(destination, NumPar) %>%
                    select(-source)
                SimDispMat[is.na(SimDispMat)] <- 0
                
                #add in unassigned row
                unassigned <- t(as.matrix(SimSampSummary2012 %>% arrange(destination) %>% ungroup() %>% select(NumUnassigned)))
                rownames(unassigned)<-NULL
                colnames(unassigned)<-names(SimDispMat)
                
                SimDispMatFull2012 <- rbind(SimDispMat, unassigned)
                colnames(SimDispMatFull2012) <- NULL

        #The full remade parentage matrix
        Assignments <- SimDispMatFull2012
    
    #NOW FIT THE KERNEL


x <- list(Distances=Distances, Assignments=Assignments, Sampled_reefs=Sampled_reefs, Reef_sizes=Reef_sizes, Adult_sample_proportions=Adult_sample_proportions) #put inputs into a list because that's the bbmle format

Sim2012Fit <- suppressWarnings(mle2(LL_kt_bbmle, start=list(k=-3, theta=1), lower=c(-10, 0.15), upper=c(10, 8), method="L-BFGS-B", data=x, control=list(maxit=500)))

BestK2012 <- as.numeric(coef(Sim2012Fit)[1])
BestTheta2012 <- as.numeric(coef(Sim2012Fit)[2])
MDD2012 <- as.numeric(cubintegrate(integrate_kernel_sum1, lower = 0, upper = Inf, k=BestK2012, theta=BestTheta2012, , method = "pcubature")$integral)
k_eval <- BestK2012
theta_eval <- BestTheta2012
Med2012  <- round(nleqslv(x = 7, fn = cdf_solve)$x, 2) 
Dist90_2012 <- round(nleqslv(x = 7, fn = cdf_solve90)$x, 2)
    
    #store the info in this df
    SimulatedKernels2012_beta <- as.data.frame(matrix(nrow=1, ncol=7), stringsAsFactors = FALSE)
    colnames(SimulatedKernels2012_beta) <- col
    
    SimulatedKernels2012_beta$year <- 2012
    SimulatedKernels2012_beta$k <- BestK2012
    SimulatedKernels2012_beta$theta <- BestTheta2012
    SimulatedKernels2012_beta$mdd <- MDD2012
    SimulatedKernels2012_beta$med <- Med2012
    SimulatedKernels2012_beta$dist90 <- Dist90_2012
    SimulatedKernels2012_beta$iteration <- n
    
    
#join results into larger df
SimulatedKernels2012 <- bind_rows(SimulatedKernels2012, SimulatedKernels2012_beta)
    
setTxtProgressBar(pb, n)


}
close(pb)
EndTime <- Sys.time()
EndTime-StartTime
options(warn=0) #turn warnings back on


write.csv(SimulatedKernels2012, file="~/oceanography/script_output/KernelFits/100SimulatedKernels2012Growth.csv", row.names=F)

  |======================================================================| 100%


Time difference of 20.23442 mins

In [29]:
#re-do ensembles with only the sampling correction
#awesome! it works! let's rip with a 100 iteration for loop
col <- c("year", "k", "theta", "mdd", "med", "dist90", "iteration")

#awesome! it works! let's rip with a 100 iteration for loop
col <- c("year", "k", "theta", "mdd", "med", "dist90", "iteration")
SimulatedKernels2012 <- as.data.frame(matrix(nrow=0, ncol=7), stringsAsFactors = FALSE)
colnames(SimulatedKernels2012) <- col

#don't print warnings for this loop, they are only for setting row names in a tibble which is depracated. But it works. 
options(warn=-1)

pb <- txtProgressBar(min = 0, max = 100, style = 3)

StartTime <- Sys.time()

for(n in 1:100){
        
    #SAMPLE THE SIMULATION DATA, then fit a kernel
    
    #make an empty dataframe with the same columns as original, this will be sampled particles
    SimulatedSampling2012 = Simulation2012[FALSE,]
    
    for(i in 1:nrow(NGenOffs2012)){ 
        
        destination_eval <- as.character(NGenOffs2012$site[i]) #pick out a destination site
        
        SimSampDestination <- Simulation2012SurvGrowth %>% #USING DF WITH SURV/SIZE WEIGHTING ***SurvGrowth
                                filter(destination==destination_eval) %>%
                                sample_n(as.numeric(NGenOffs2012$n_offs_gen[i]),  replace=F) %>% #weight=SurvivalWeightLin,, sample particles that landed at that destination, number corresponding to the actual number sampled at that site in this year
                                select(one_of(names(Simulation2012)))#keep matching columns with the original df instead of carrying over the columns used for survival weighting
        
        SimulatedSampling2012 <- bind_rows(SimulatedSampling2012, SimSampDestination) #build into a sampled particle df
    }
    
    #assign a numeric ID for each row (which is a sampled particle)
    SimulatedSampling2012 <- SimulatedSampling2012 %>%
        mutate(ParticleSampID=paste ("P", row_number(), sep = "", collapse = NULL))
    
    #nrow(SimulatedSampling2012)==sum(NGenOffs2012$n_offs_gen) #should be TRUE
    
    #now randomly assign parentage match status to some of these rows
    NumPar <- as.numeric(kernels %>%
        filter(Year=="2012") %>%
        select(NumParentageMatches))
    
    SimulatedSampling2012Par <- SimulatedSampling2012 %>%
        ungroup() %>%
        filter(source %in% NGenOffs2012$site) %>% #we can only assign parentage if we sampled the source site, we've already ensured that we sampled the destination site in the for loop by indexing destination sites to evaluate 
        sample_n(NumPar, replace=F) %>% #sample number of rows consitent with numbers of parentage matches
        mutate(Parentage=1) #%>% #assign these rows a positive match value
        #arrange(date, source, destination) %>%#arrange by the same order as the orginal df, then bind columns together to avoid unexpected repeats from a join
        #select(date, source, destination, Parentage) #drop columns that will result in repeats 
    
    #find the rows that haven't been assigned parentage
    SimulatedSampling2012_2 <- anti_join((SimulatedSampling2012 %>% ungroup()), SimulatedSampling2012Par, by=c("ParticleSampID")) 
    
    SimulatedSampling2012_2$Parentage <- 0 #add in the column for parentage
    
    #combine parentage and non-parentage dfs
    SimulatedSampling2012Par3 <- bind_rows(SimulatedSampling2012Par, SimulatedSampling2012_2) %>%
        mutate(YearSampled=as.character(YearSampled))
    
    
    #NOW FORMAT INTO PARENTAGE MATRIX
    
                PropSamp2012 <- PropSamp %>%  #add in all sampled sites as possible parent sites
                            filter(end_year =="2012" & PropSamp > 0) %>%
                            #rename(source="site") %>%
                            select(-end_year, -PropSamp)

                #PropSamp2012$destination <- PropSamp2012$source #make another column for destination
                
                #make a table with the total number of particles sampled at each site
                TotalSimSamp <- SimulatedSampling2012Par3 %>% 
                            select(source, destination) %>%
                            group_by(destination) %>%
                            summarise(NumSimSampRec=n()) %>%
                            ungroup()
               
                
                #add in all surveyed sites
                TotalSimSamp2 <- left_join(PropSamp2012, TotalSimSamp, by=c(site="destination")) %>% #join sites to destination, which will add a row for the surveyed sites with no samples
                    mutate(NumSimSampRec=ifelse(is.na(NumSimSampRec), 0, NumSimSampRec)) %>% #change the Na's that result to 0s, because we didn't get samples there
                    rename(destination="site")#rename sites to destination after joining
                
                
                
                #make a table with the total number of parentage matches found in the simulated loop
                TotalSimPar <- SimulatedSampling2012Par3 %>% 
                            select(source, destination, Parentage) %>%
                            group_by(source,destination) %>%
                            mutate(NumPar=sum(Parentage)) %>% #sum all the parentage observations along each ROUTE
                            select(-Parentage) %>%
                            ungroup() %>%
                            distinct(source, destination, .keep_all = T)
                
                
                #combine for a summary table BY DESTINATION, this below will be for UNASSIGNED ROWS
                SimSampSummary2012 <- left_join(TotalSimSamp2, TotalSimPar, by="destination") %>%
                    distinct(source, destination, .keep_all = T) %>%
                    group_by(destination) %>%
                    mutate(NumPar=ifelse(is.na(NumPar), 0, NumPar)) %>% #change the Na's that result to 0s, because we didn't get samples there
                    mutate(NumPar=sum(NumPar)) %>% #sum all the parentage observations at each DESTINATION
                    distinct(destination, .keep_all = T) %>%
                    select(-source) %>%
                    mutate(NumUnassigned=NumSimSampRec-NumPar)
                                
                
                #finally, join all of the sites surveyed for a complete parentage matrix
                #for this, need to have a source/destination column for all sites
                PropSamp2012 <- PropSamp2012 %>%
                                rename(source="site")
                
                PropSamp2012$destination <- PropSamp2012$source #make another column for destination
                
                #this below will become parentage matrix! 
                TotalSimPar2 <- full_join(TotalSimPar,PropSamp2012, by=c("source", "destination")) %>%
                    mutate(NumPar=ifelse(is.na(NumPar), 0, NumPar))
        
                
                #spread into matrix format
                SimDispMat <- TotalSimPar2 %>%
                    filter(source %in% NGenOffs2012$site) %>%
                    arrange(source, destination)  %>%
                    spread(destination, NumPar) %>%
                    select(-source)
                SimDispMat[is.na(SimDispMat)] <- 0
                
                #add in unassigned row
                unassigned <- t(as.matrix(SimSampSummary2012 %>% arrange(destination) %>% ungroup() %>% select(NumUnassigned)))
                rownames(unassigned)<-NULL
                colnames(unassigned)<-names(SimDispMat)
                
                SimDispMatFull2012 <- rbind(SimDispMat, unassigned)
                colnames(SimDispMatFull2012) <- NULL

        #The full remade parentage matrix
        Assignments <- SimDispMatFull2012
    
    #NOW FIT THE KERNEL


x <- list(Distances=Distances, Assignments=Assignments, Sampled_reefs=Sampled_reefs, Reef_sizes=Reef_sizes, Adult_sample_proportions=Adult_sample_proportions) #put inputs into a list because that's the bbmle format

Sim2012Fit <- suppressWarnings(mle2(LL_kt_bbmle, start=list(k=-3, theta=1), lower=c(-10, 0.15), upper=c(10, 8), method="L-BFGS-B", data=x, control=list(maxit=500)))

BestK2012 <- as.numeric(coef(Sim2012Fit)[1])
BestTheta2012 <- as.numeric(coef(Sim2012Fit)[2])
MDD2012 <- as.numeric(cubintegrate(integrate_kernel_sum1, lower = 0, upper = Inf, k=BestK2012, theta=BestTheta2012, , method = "pcubature")$integral)
k_eval <- BestK2012
theta_eval <- BestTheta2012
Med2012  <- round(nleqslv(x = 7, fn = cdf_solve)$x, 2) 
Dist90_2012 <- round(nleqslv(x = 7, fn = cdf_solve90)$x, 2)
    
    #store the info in this df
    SimulatedKernels2012_beta <- as.data.frame(matrix(nrow=1, ncol=7), stringsAsFactors = FALSE)
    colnames(SimulatedKernels2012_beta) <- col
    
    SimulatedKernels2012_beta$year <- 2012
    SimulatedKernels2012_beta$k <- BestK2012
    SimulatedKernels2012_beta$theta <- BestTheta2012
    SimulatedKernels2012_beta$mdd <- MDD2012
    SimulatedKernels2012_beta$med <- Med2012
    SimulatedKernels2012_beta$dist90 <- Dist90_2012
    SimulatedKernels2012_beta$iteration <- n
    
    
#join results into larger df
SimulatedKernels2012 <- bind_rows(SimulatedKernels2012, SimulatedKernels2012_beta)
    
setTxtProgressBar(pb, n)


}
close(pb)
EndTime <- Sys.time()
EndTime-StartTime
options(warn=0) #turn warnings back on


write.csv(SimulatedKernels2012, file="~/oceanography/script_output/KernelFits/100SimulatedKernels2012Growth.csv", row.names=F)
SimulatedKernels2013 <- as.data.frame(matrix(nrow=0, ncol=7), stringsAsFactors = FALSE)
colnames(SimulatedKernels2013) <- col

#don't print warnings for this loop, they are only for setting row names in a tibble which is depracated. But it works. 
options(warn=-1)

pb <- txtProgressBar(min = 0, max = 100, style = 3)

StartTime <- Sys.time()

for(n in 1:100){
        
    #SAMPLE THE SIMULATION DATA, then fit a kernel
    
    #make an empty dataframe with the same columns as original, this will be sampled particles
    SimulatedSampling2013 = Simulation2013[FALSE,]
    
    for(i in 1:nrow(NGenOffs2013)){ 
        
        destination_eval <- as.character(NGenOffs2013$site[i]) #pick out a destination site
        
        SimSampDestination <- Simulation2013SurvGrowth %>% #USING DF WITH SURV/SIZE WEIGHTING ***SurvGrowth
                                filter(destination==destination_eval) %>%
                                sample_n(as.numeric(NGenOffs2013$n_offs_gen[i]),  replace=F) %>% #weight=SurvivalWeightLin,, sample particles that landed at that destination, number corresponding to the actual number sampled at that site in this year
                                select(one_of(names(Simulation2013)))#keep matching columns with the original df instead of carrying over the columns used for survival weighting
        
        SimulatedSampling2013 <- bind_rows(SimulatedSampling2013, SimSampDestination) #build into a sampled particle df
    }
    
    #assign a numeric ID for each row (which is a sampled particle)
    SimulatedSampling2013 <- SimulatedSampling2013 %>%
        mutate(ParticleSampID=paste ("P", row_number(), sep = "", collapse = NULL))
    
    #nrow(SimulatedSampling2013)==sum(NGenOffs2013$n_offs_gen) #should be TRUE
    
    #now randomly assign parentage match status to some of these rows
    NumPar <- as.numeric(kernels %>%
        filter(Year=="2013") %>%
        select(NumParentageMatches))
    
    SimulatedSampling2013Par <- SimulatedSampling2013 %>%
        ungroup() %>%
        filter(source %in% NGenOffs2013$site) %>% #we can only assign parentage if we sampled the source site, we've already ensured that we sampled the destination site in the for loop by indexing destination sites to evaluate 
        sample_n(NumPar, replace=F) %>% #sample number of rows consitent with numbers of parentage matches
        mutate(Parentage=1) #%>% #assign these rows a positive match value
        #arrange(date, source, destination) %>%#arrange by the same order as the orginal df, then bind columns together to avoid unexpected repeats from a join
        #select(date, source, destination, Parentage) #drop columns that will result in repeats 
    
    #find the rows that haven't been assigned parentage
    SimulatedSampling2013_2 <- anti_join((SimulatedSampling2013 %>% ungroup()), SimulatedSampling2013Par, by=c("ParticleSampID")) 
    
    SimulatedSampling2013_2$Parentage <- 0 #add in the column for parentage
    
    #combine parentage and non-parentage dfs
    SimulatedSampling2013Par3 <- bind_rows(SimulatedSampling2013Par, SimulatedSampling2013_2) %>%
        mutate(YearSampled=as.character(YearSampled))
    
    
    #NOW FORMAT INTO PARENTAGE MATRIX
    
                PropSamp2013 <- PropSamp %>%  #add in all sampled sites as possible parent sites
                            filter(end_year =="2013" & PropSamp > 0) %>%
                            #rename(source="site") %>%
                            select(-end_year, -PropSamp)

                #PropSamp2013$destination <- PropSamp2013$source #make another column for destination
                
                #make a table with the total number of particles sampled at each site
                TotalSimSamp <- SimulatedSampling2013Par3 %>% 
                            select(source, destination) %>%
                            group_by(destination) %>%
                            summarise(NumSimSampRec=n()) %>%
                            ungroup()
               
                
                #add in all surveyed sites
                TotalSimSamp2 <- left_join(PropSamp2013, TotalSimSamp, by=c(site="destination")) %>% #join sites to destination, which will add a row for the surveyed sites with no samples
                    mutate(NumSimSampRec=ifelse(is.na(NumSimSampRec), 0, NumSimSampRec)) %>% #change the Na's that result to 0s, because we didn't get samples there
                    rename(destination="site")#rename sites to destination after joining
                
                
                
                #make a table with the total number of parentage matches found in the simulated loop
                TotalSimPar <- SimulatedSampling2013Par3 %>% 
                            select(source, destination, Parentage) %>%
                            group_by(source,destination) %>%
                            mutate(NumPar=sum(Parentage)) %>% #sum all the parentage observations along each ROUTE
                            select(-Parentage) %>%
                            ungroup() %>%
                            distinct(source, destination, .keep_all = T)
                
                
                #combine for a summary table BY DESTINATION, this below will be for UNASSIGNED ROWS
                SimSampSummary2013 <- left_join(TotalSimSamp2, TotalSimPar, by="destination") %>%
                    distinct(source, destination, .keep_all = T) %>%
                    group_by(destination) %>%
                    mutate(NumPar=ifelse(is.na(NumPar), 0, NumPar)) %>% #change the Na's that result to 0s, because we didn't get samples there
                    mutate(NumPar=sum(NumPar)) %>% #sum all the parentage observations at each DESTINATION
                    distinct(destination, .keep_all = T) %>%
                    select(-source) %>%
                    mutate(NumUnassigned=NumSimSampRec-NumPar)
                                
                
                #finally, join all of the sites surveyed for a complete parentage matrix
                #for this, need to have a source/destination column for all sites
                PropSamp2013 <- PropSamp2013 %>%
                                rename(source="site")
                
                PropSamp2013$destination <- PropSamp2013$source #make another column for destination
                
                #this below will become parentage matrix! 
                TotalSimPar2 <- full_join(TotalSimPar,PropSamp2013, by=c("source", "destination")) %>%
                    mutate(NumPar=ifelse(is.na(NumPar), 0, NumPar))
        
                
                #spread into matrix format
                SimDispMat <- TotalSimPar2 %>%
                    filter(source %in% NGenOffs2013$site) %>%
                    arrange(source, destination)  %>%
                    spread(destination, NumPar) %>%
                    select(-source)
                SimDispMat[is.na(SimDispMat)] <- 0
                
                #add in unassigned row
                unassigned <- t(as.matrix(SimSampSummary2013 %>% arrange(destination) %>% ungroup() %>% select(NumUnassigned)))
                rownames(unassigned)<-NULL
                colnames(unassigned)<-names(SimDispMat)
                
                SimDispMatFull2013 <- rbind(SimDispMat, unassigned)
                colnames(SimDispMatFull2013) <- NULL

        #The full remade parentage matrix
        Assignments <- SimDispMatFull2013
    
    #NOW FIT THE KERNEL


x <- list(Distances=Distances, Assignments=Assignments, Sampled_reefs=Sampled_reefs, Reef_sizes=Reef_sizes, Adult_sample_proportions=Adult_sample_proportions) #put inputs into a list because that's the bbmle format

Sim2013Fit <- suppressWarnings(mle2(LL_kt_bbmle, start=list(k=-3, theta=1), lower=c(-10, 0.15), upper=c(10, 8), method="L-BFGS-B", data=x, control=list(maxit=500)))

BestK2013 <- as.numeric(coef(Sim2013Fit)[1])
BestTheta2013 <- as.numeric(coef(Sim2013Fit)[2])
MDD2013 <- as.numeric(cubintegrate(integrate_kernel_sum1, lower = 0, upper = Inf, k=BestK2013, theta=BestTheta2013, , method = "pcubature")$integral)
k_eval <- BestK2013
theta_eval <- BestTheta2013
Med2013  <- round(nleqslv(x = 7, fn = cdf_solve)$x, 2) 
Dist90_2013 <- round(nleqslv(x = 7, fn = cdf_solve90)$x, 2)
    
    #store the info in this df
    SimulatedKernels2013_beta <- as.data.frame(matrix(nrow=1, ncol=7), stringsAsFactors = FALSE)
    colnames(SimulatedKernels2013_beta) <- col
    
    SimulatedKernels2013_beta$year <- 2013
    SimulatedKernels2013_beta$k <- BestK2013
    SimulatedKernels2013_beta$theta <- BestTheta2013
    SimulatedKernels2013_beta$mdd <- MDD2013
    SimulatedKernels2013_beta$med <- Med2013
    SimulatedKernels2013_beta$dist90 <- Dist90_2013
    SimulatedKernels2013_beta$iteration <- n
    
    
#join results into larger df
SimulatedKernels2013 <- bind_rows(SimulatedKernels2013, SimulatedKernels2013_beta)
    
setTxtProgressBar(pb, n)


}
close(pb)
EndTime <- Sys.time()
EndTime-StartTime
options(warn=0) #turn warnings back on


write.csv(SimulatedKernels2013, file="~/oceanography/script_output/KernelFits/100SimulatedKernels2013Growth.csv", row.names=F)

SimulatedKernels2014 <- as.data.frame(matrix(nrow=0, ncol=7), stringsAsFactors = FALSE)
colnames(SimulatedKernels2014) <- col

#don't print warnings for this loop, they are only for setting row names in a tibble which is depracated. But it works. 
options(warn=-1)

pb <- txtProgressBar(min = 0, max = 100, style = 3)

StartTime <- Sys.time()

for(n in 1:100){
        
    #SAMPLE THE SIMULATION DATA, then fit a kernel
    
    #make an empty dataframe with the same columns as original, this will be sampled particles
    SimulatedSampling2014 = Simulation2014[FALSE,]
    
    for(i in 1:nrow(NGenOffs2014)){ 
        
        destination_eval <- as.character(NGenOffs2014$site[i]) #pick out a destination site
        
        SimSampDestination <- Simulation2014SurvGrowth %>% #USING DF WITH SURV/SIZE WEIGHTING ***SurvGrowth
                                filter(destination==destination_eval) %>%
                                sample_n(as.numeric(NGenOffs2014$n_offs_gen[i]),  replace=F) %>% #weight=SurvivalWeightLin,, sample particles that landed at that destination, number corresponding to the actual number sampled at that site in this year
                                select(one_of(names(Simulation2014)))#keep matching columns with the original df instead of carrying over the columns used for survival weighting
        
        SimulatedSampling2014 <- bind_rows(SimulatedSampling2014, SimSampDestination) #build into a sampled particle df
    }
    
    #assign a numeric ID for each row (which is a sampled particle)
    SimulatedSampling2014 <- SimulatedSampling2014 %>%
        mutate(ParticleSampID=paste ("P", row_number(), sep = "", collapse = NULL))
    
    #nrow(SimulatedSampling2014)==sum(NGenOffs2014$n_offs_gen) #should be TRUE
    
    #now randomly assign parentage match status to some of these rows
    NumPar <- as.numeric(kernels %>%
        filter(Year=="2014") %>%
        select(NumParentageMatches))
    
    SimulatedSampling2014Par <- SimulatedSampling2014 %>%
        ungroup() %>%
        filter(source %in% NGenOffs2014$site) %>% #we can only assign parentage if we sampled the source site, we've already ensured that we sampled the destination site in the for loop by indexing destination sites to evaluate 
        sample_n(NumPar, replace=F) %>% #sample number of rows consitent with numbers of parentage matches
        mutate(Parentage=1) #%>% #assign these rows a positive match value
        #arrange(date, source, destination) %>%#arrange by the same order as the orginal df, then bind columns together to avoid unexpected repeats from a join
        #select(date, source, destination, Parentage) #drop columns that will result in repeats 
    
    #find the rows that haven't been assigned parentage
    SimulatedSampling2014_2 <- anti_join((SimulatedSampling2014 %>% ungroup()), SimulatedSampling2014Par, by=c("ParticleSampID")) 
    
    SimulatedSampling2014_2$Parentage <- 0 #add in the column for parentage
    
    #combine parentage and non-parentage dfs
    SimulatedSampling2014Par3 <- bind_rows(SimulatedSampling2014Par, SimulatedSampling2014_2) %>%
        mutate(YearSampled=as.character(YearSampled))
    
    
    #NOW FORMAT INTO PARENTAGE MATRIX
    
                PropSamp2014 <- PropSamp %>%  #add in all sampled sites as possible parent sites
                            filter(end_year =="2014" & PropSamp > 0) %>%
                            #rename(source="site") %>%
                            select(-end_year, -PropSamp)

                #PropSamp2014$destination <- PropSamp2014$source #make another column for destination
                
                #make a table with the total number of particles sampled at each site
                TotalSimSamp <- SimulatedSampling2014Par3 %>% 
                            select(source, destination) %>%
                            group_by(destination) %>%
                            summarise(NumSimSampRec=n()) %>%
                            ungroup()
               
                
                #add in all surveyed sites
                TotalSimSamp2 <- left_join(PropSamp2014, TotalSimSamp, by=c(site="destination")) %>% #join sites to destination, which will add a row for the surveyed sites with no samples
                    mutate(NumSimSampRec=ifelse(is.na(NumSimSampRec), 0, NumSimSampRec)) %>% #change the Na's that result to 0s, because we didn't get samples there
                    rename(destination="site")#rename sites to destination after joining
                
                
                
                #make a table with the total number of parentage matches found in the simulated loop
                TotalSimPar <- SimulatedSampling2014Par3 %>% 
                            select(source, destination, Parentage) %>%
                            group_by(source,destination) %>%
                            mutate(NumPar=sum(Parentage)) %>% #sum all the parentage observations along each ROUTE
                            select(-Parentage) %>%
                            ungroup() %>%
                            distinct(source, destination, .keep_all = T)
                
                
                #combine for a summary table BY DESTINATION, this below will be for UNASSIGNED ROWS
                SimSampSummary2014 <- left_join(TotalSimSamp2, TotalSimPar, by="destination") %>%
                    distinct(source, destination, .keep_all = T) %>%
                    group_by(destination) %>%
                    mutate(NumPar=ifelse(is.na(NumPar), 0, NumPar)) %>% #change the Na's that result to 0s, because we didn't get samples there
                    mutate(NumPar=sum(NumPar)) %>% #sum all the parentage observations at each DESTINATION
                    distinct(destination, .keep_all = T) %>%
                    select(-source) %>%
                    mutate(NumUnassigned=NumSimSampRec-NumPar)
                                
                
                #finally, join all of the sites surveyed for a complete parentage matrix
                #for this, need to have a source/destination column for all sites
                PropSamp2014 <- PropSamp2014 %>%
                                rename(source="site")
                
                PropSamp2014$destination <- PropSamp2014$source #make another column for destination
                
                #this below will become parentage matrix! 
                TotalSimPar2 <- full_join(TotalSimPar,PropSamp2014, by=c("source", "destination")) %>%
                    mutate(NumPar=ifelse(is.na(NumPar), 0, NumPar))
        
                
                #spread into matrix format
                SimDispMat <- TotalSimPar2 %>%
                    filter(source %in% NGenOffs2014$site) %>%
                    arrange(source, destination)  %>%
                    spread(destination, NumPar) %>%
                    select(-source)
                SimDispMat[is.na(SimDispMat)] <- 0
                
                #add in unassigned row
                unassigned <- t(as.matrix(SimSampSummary2014 %>% arrange(destination) %>% ungroup() %>% select(NumUnassigned)))
                rownames(unassigned)<-NULL
                colnames(unassigned)<-names(SimDispMat)
                
                SimDispMatFull2014 <- rbind(SimDispMat, unassigned)
                colnames(SimDispMatFull2014) <- NULL

        #The full remade parentage matrix
        Assignments <- SimDispMatFull2014
    
    #NOW FIT THE KERNEL


x <- list(Distances=Distances, Assignments=Assignments, Sampled_reefs=Sampled_reefs, Reef_sizes=Reef_sizes, Adult_sample_proportions=Adult_sample_proportions) #put inputs into a list because that's the bbmle format

Sim2014Fit <- suppressWarnings(mle2(LL_kt_bbmle, start=list(k=-3, theta=1), lower=c(-10, 0.15), upper=c(10, 8), method="L-BFGS-B", data=x, control=list(maxit=500)))

BestK2014 <- as.numeric(coef(Sim2014Fit)[1])
BestTheta2014 <- as.numeric(coef(Sim2014Fit)[2])
MDD2014 <- as.numeric(cubintegrate(integrate_kernel_sum1, lower = 0, upper = Inf, k=BestK2014, theta=BestTheta2014, , method = "pcubature")$integral)
k_eval <- BestK2014
theta_eval <- BestTheta2014
Med2014  <- round(nleqslv(x = 7, fn = cdf_solve)$x, 2) 
Dist90_2014 <- round(nleqslv(x = 7, fn = cdf_solve90)$x, 2)
    
    #store the info in this df
    SimulatedKernels2014_beta <- as.data.frame(matrix(nrow=1, ncol=7), stringsAsFactors = FALSE)
    colnames(SimulatedKernels2014_beta) <- col
    
    SimulatedKernels2014_beta$year <- 2014
    SimulatedKernels2014_beta$k <- BestK2014
    SimulatedKernels2014_beta$theta <- BestTheta2014
    SimulatedKernels2014_beta$mdd <- MDD2014
    SimulatedKernels2014_beta$med <- Med2014
    SimulatedKernels2014_beta$dist90 <- Dist90_2014
    SimulatedKernels2014_beta$iteration <- n
    
    
#join results into larger df
SimulatedKernels2014 <- bind_rows(SimulatedKernels2014, SimulatedKernels2014_beta)
    
setTxtProgressBar(pb, n)


}
close(pb)
EndTime <- Sys.time()
EndTime-StartTime
options(warn=0) #turn warnings back on


write.csv(SimulatedKernels2014, file="~/oceanography/script_output/KernelFits/100SimulatedKernels2014Growth.csv", row.names=F)



  |======================================================================| 100%


Time difference of 9.855328 mins

  |======================================================================| 100%


Time difference of 14.26825 mins

  |======================================================================| 100%


Time difference of 11.84184 mins

In [147]:
#combine simulations
SimulationAll <- bind_rows(Simulation2012, Simulation2013, Simulation2014)

#make a DF of all of the sampled fish over 2012-2014
AllYearsRecruits <- AnnualRecsSamp %>% distinct(site, .keep_all = T)
sum(AllYearsRecruits$n_offs_gen_all_years) #should be 394 for 2012-2014

[1] 394

In [152]:
#what if we account for post-settlement mortality and not sampling <3.5 cm recruits?
NumDays <- nrow(SimulationAll %>% distinct(date))#200 days in a simulation

#make a df with weighted values for the days of the simluation oldest to most recent
SimDates <- SimulationAll %>%
    distinct(date) %>%
    mutate(DateSeq=row_number()) %>% #assign row numbers, so more recent dates are higher than older
    mutate(SurvivalWeightExp=exp(DateSeq)) %>% #apply the exponential function to get a survival column
    mutate(SurvivalWeightExp=SurvivalWeightExp/max(SurvivalWeightExp))%>%#normalize as a survival "probability"
    mutate(SurvivalWeightLin=(((0.25/2)/5)*DateSeq)) %>%#maybe a linear survival function (y=mx)is more appropriate (eyeball from Allison's Fig 3C because what the hell why not we're just thinking here)
    mutate(SurvivalWeightLin=SurvivalWeightLin/max(SurvivalWeightLin)) #%>%#normalize as a survival "probability"
    #filter(DateSeq %!in% seq((max(DateSeq)-60), max(DateSeq), 1))#account for the fact that we didn't sample <3.5 cm
#the max survival prob will be different depending on when I filter out the more recent recruits. I don't want an artificially high prob of sampling based on a survival prob of 1 so I'm cutting fish out after prob weight calculations
SimulationAllSurvGrowth <- left_join(SimulationAll, SimDates, by="date") %>%
    filter(DateSeq %!in% seq((max(DateSeq)-60), max(DateSeq), 1))
nrow(SimulationAllSurvGrowth)<nrow(SimulationAll)#should be TRUE

[1] TRUE

In [155]:
#get together kernel fitting components for all years/seasonal

#assemble all of the components for annual kernel fitting
centroids <- read.csv("~/oceanography/empirical_data/site_centroids_simulation_kernels.csv", header=TRUE)

Area <- read.csv("~/oceanography/empirical_data/site_area_header_nonsurveyed_simulation_kernels.csv", header=TRUE) %>%
    arrange(site) %>%
    filter(site %!in% c("near_north_full1", "near_north_full2", "near_north_full3", "near_south_full1", "near_south_full2", "near_south_full3")) %>%
    mutate(kmsq=msq*10^-6) %>%
    select(kmsq)
Reef_sizes <- as.matrix(Area)

#give every site in the distance matrix of the simulation (even if we didn't sample there) an index number
AllSites <- centroids %>%
    select(site) %>%
    arrange(site)
nrow(AllSites) #should be 34x1
AllSites$index <- seq(from=1, to=34, by=1)

PropSampAll <- PropSamp %>%  #add in all sampled sites as possible parent sites
            filter(end_year =="2014" & PropSamp > 0) %>%
            rename(source="site") %>%
            select(-end_year, -PropSamp)

SampledSitesAll <- inner_join(PropSampAll %>% select(source), AllSites, by=c(source="site")) 

#check for correct number of rows
nrow(SampledSitesAll)==nrow(PropSampAll) #should be true
SampledSitesAllIndex <- t(as.matrix(SampledSitesAll$index))
ncol(SampledSitesAllIndex)

Sampled_reefs <- SampledSitesAllIndex

#proportion sampled matrix for kernel fitting

Adult_sample_proportions <- PropSamp %>%  #add in all sampled sites as possible parent sites
                filter(end_year =="2014" & PropSamp > 0) 

Adult_sample_proportions <- as.matrix(Adult_sample_proportions$PropSamp)
nrow(Adult_sample_proportions) 

#distance matrix using the centroids with combined Magbangon
### List of source locations
sites_source <- centroids

### List of destination locations
sites_dest <- centroids

dist_mat_m <- distm(sites_source[,c('lon','lat')], sites_source[,c('lon','lat')], fun=distVincentyEllipsoid)
Distances <- dist_mat_m*10^-3

Centroids <- centroids %>%
    select(-site)

#for the all years sites sampled in the simulation, it's a total of 16. That's lower than the all year used in kernel fitting because we didn't sample Sitio Tugas or Gabas until after 2014. We also combined the Magbangons, so 19-16=3 accounted for

[1] 34

[1] TRUE

[1] 16

[1] 16

In [ ]:
#All of the corresponding simulation years combined (2011-14)

col <- c("year", "k", "theta", "mdd", "med", "dist90", "iteration")
SimulatedKernelsAll <- as.data.frame(matrix(nrow=0, ncol=7), stringsAsFactors = FALSE)
colnames(SimulatedKernelsAll) <- col

#don't print warnings for this loop, they are only for setting row names in a tibble which is depracated. But it works. 
options(warn=-1)

pb <- txtProgressBar(min = 0, max = 100, style = 3)#7 years in each interation

StartTime <- Sys.time()


for(n in 1:100){
        
    #SAMPLE THE SIMULATION DATA, then fit a kernel
    
    SimulatedSamplingAll = SimulationAll[FALSE,]
    
    for(i in 1:nrow(AllYearsRecruits)){ 
        
        destination_eval <- as.character(AllYearsRecruits$site[i]) #pick out a destination site
        
        SimSampDestination <- SimulationAllSurvGrowth %>%
                                    filter(destination==destination_eval) %>%
                                    sample_n(as.numeric(AllYearsRecruits$n_offs_gen_all_years[i]), replace=F) #sample particles that landed at that destination, number corresponding to the actual number sampled at that site in this year
        
        SimulatedSamplingAll <- bind_rows(SimulatedSamplingAll, SimSampDestination) #build into a sampled particle df
    }

    #assign a numeric ID for each row (which is a sampled particle)
    SimulatedSamplingAll <- SimulatedSamplingAll %>%
        mutate(ParticleSampID=paste ("P", row_number(), sep = "", collapse = NULL))
    
    #nrow(SimulatedSamplingAll)==sum(NGenOffsAll$n_offs_gen) #should be TRUE
    
    #now randomly assign parentage match status to some of these rows
    NumPar <- as.numeric(kernels %>% #should be 37 for 2012-2014
        filter(Year %in% c("2012", "2013", "2014")) %>%
        select(NumParentageMatches) %>%
        summarise(NumParentageMatches=sum(NumParentageMatches)))
    
    SimulatedSamplingAllPar <- SimulatedSamplingAll %>%
        ungroup() %>%
        filter(source %in% NGenOffsAll$site) %>% #we can only assign parentage if we sampled the source site, we've already ensured that we sampled the destination site in the for loop by indexing destination sites to evaluate 
        sample_n(NumPar, replace=F) %>% #sample number of rows consitent with numbers of parentage matches
        mutate(Parentage=1) #%>% #assign these rows a positive match value
        #arrange(date, source, destination) %>%#arrange by the same order as the orginal df, then bind columns together to avoid unexpected repeats from a join
        #select(date, source, destination, Parentage) #drop columns that will result in repeats 
    
    #find the rows that haven't been assigned parentage
    SimulatedSamplingAll_2 <- anti_join((SimulatedSamplingAll %>% ungroup()), SimulatedSamplingAllPar, by=c("ParticleSampID")) 
    
    SimulatedSamplingAll_2$Parentage <- 0 #add in the column for parentage
    
    #combine parentage and non-parentage dfs
    SimulatedSamplingAllPar3 <- bind_rows(SimulatedSamplingAllPar, SimulatedSamplingAll_2) %>%
        mutate(YearSampled=as.character(YearSampled))
    
    
    #NOW FORMAT INTO PARENTAGE MATRIX
    
                PropSampAll <- PropSamp %>%  #add in all sampled sites as possible parent sites
                            filter(end_year =="2014" & PropSamp > 0) %>%
                            #rename(source="site") %>%
                            select(-end_year, -PropSamp)

                #PropSampAll$destination <- PropSampAll$source #make another column for destination
                
                #make a table with the total number of particles sampled at each site
                TotalSimSamp <- SimulatedSamplingAllPar3 %>% 
                            select(source, destination) %>%
                            group_by(destination) %>%
                            summarise(NumSimSampRec=n()) %>%
                            ungroup()
               
                
                #add in all surveyed sites
                TotalSimSamp2 <- left_join(PropSampAll, TotalSimSamp, by=c(site="destination")) %>% #join sites to destination, which will add a row for the surveyed sites with no samples
                    mutate(NumSimSampRec=ifelse(is.na(NumSimSampRec), 0, NumSimSampRec)) %>% #change the Na's that result to 0s, because we didn't get samples there
                    rename(destination="site")#rename sites to destination after joining
                
                
                
                #make a table with the total number of parentage matches found in the simulated loop
                TotalSimPar <- SimulatedSamplingAllPar3 %>% 
                            select(source, destination, Parentage) %>%
                            group_by(source,destination) %>%
                            mutate(NumPar=sum(Parentage)) %>% #sum all the parentage observations along each ROUTE
                            select(-Parentage) %>%
                            ungroup() %>%
                            distinct(source, destination, .keep_all = T)
                
                
                #combine for a summary table BY DESTINATION, this below will be for UNASSIGNED ROWS
                SimSampSummaryAll <- left_join(TotalSimSamp2, TotalSimPar, by="destination") %>%
                    distinct(source, destination, .keep_all = T) %>%
                    group_by(destination) %>%
                    mutate(NumPar=ifelse(is.na(NumPar), 0, NumPar)) %>% #change the Na's that result to 0s, because we didn't get samples there
                    mutate(NumPar=sum(NumPar)) %>% #sum all the parentage observations at each DESTINATION
                    distinct(destination, .keep_all = T) %>%
                    select(-source) %>%
                    mutate(NumUnassigned=NumSimSampRec-NumPar)
                                
                
                #finally, join all of the sites surveyed for a complete parentage matrix
                #for this, need to have a source/destination column for all sites
                PropSampAll <- PropSampAll %>%
                                rename(source="site")
                
                PropSampAll$destination <- PropSampAll$source #make another column for destination
                
                #this below will become parentage matrix! 
                TotalSimPar2 <- full_join(TotalSimPar,PropSampAll, by=c("source", "destination")) %>%
                    mutate(NumPar=ifelse(is.na(NumPar), 0, NumPar))
        
                
                #spread into matrix format
                SimDispMat <- TotalSimPar2 %>%
                    filter(source %in% NGenOffsAll$site) %>%
                    arrange(source, destination)  %>%
                    spread(destination, NumPar) %>%
                    select(-source)
                SimDispMat[is.na(SimDispMat)] <- 0
                
                #add in unassigned row
                unassigned <- t(as.matrix(SimSampSummaryAll %>% arrange(destination) %>% ungroup() %>% select(NumUnassigned)))
                rownames(unassigned)<-NULL
                colnames(unassigned)<-names(SimDispMat)
                
                SimDispMatFullAll <- rbind(SimDispMat, unassigned)
                colnames(SimDispMatFullAll) <- NULL

        #The full remade parentage matrix
        Assignments <- SimDispMatFullAll
    
    #NOW FIT THE KERNEL


x <- list(Distances=Distances, Assignments=Assignments, Sampled_reefs=Sampled_reefs, Reef_sizes=Reef_sizes, Adult_sample_proportions=Adult_sample_proportions) #put inputs into a list because that's the bbmle format

SimAllFit <- suppressWarnings(mle2(LL_kt_bbmle, start=list(k=-3, theta=1), lower=c(-10, 0.15), upper=c(10, 8), method="L-BFGS-B", data=x, control=list(maxit=500)))

BestKAll <- as.numeric(coef(SimAllFit)[1])
BestThetaAll <- as.numeric(coef(SimAllFit)[2])
MDDAll <- as.numeric(cubintegrate(integrate_kernel_sum1, lower = 0, upper = Inf, k=BestKAll, theta=BestThetaAll, , method = "pcubature")$integral)
k_eval <- BestKAll
theta_eval <- BestThetaAll
MedAll  <- round(nleqslv(x = 7, fn = cdf_solve)$x, 2) 
Dist90_All <- round(nleqslv(x = 7, fn = cdf_solve90)$x, 2)
    
    #store the info in this df
    SimulatedKernelsAll_beta <- as.data.frame(matrix(nrow=1, ncol=7), stringsAsFactors = FALSE)
    colnames(SimulatedKernelsAll_beta) <- col
    
    SimulatedKernelsAll_beta$year <- NA
    SimulatedKernelsAll_beta$k <- BestKAll
    SimulatedKernelsAll_beta$theta <- BestThetaAll
    SimulatedKernelsAll_beta$med <- MedAll
    SimulatedKernelsAll_beta$mdd <- MDDAll
    SimulatedKernelsAll_beta$dist90 <- Dist90_All
    SimulatedKernelsAll_beta$iteration <- n
    
    
#join results into larger df
SimulatedKernelsAll <- bind_rows(SimulatedKernelsAll, SimulatedKernelsAll_beta)
    
setTxtProgressBar(pb, n)


}
close(pb)
EndTime <- Sys.time()
EndTime-StartTime
options(warn=0) #turn warnings back on


#write.csv(SimulatedKernelsAll, file="~/oceanography/script_output/KernelFits/100SimulatedKernelsAllGrowth.csv", row.names=F)

In [153]:

        
    #SAMPLE THE SIMULATION DATA, then fit a kernel
    
    SimulatedSamplingAll = SimulationAll[FALSE,]
    
    for(i in 1:nrow(AllYearsRecruits)){ 
        
        destination_eval <- as.character(AllYearsRecruits$site[i]) #pick out a destination site
        
        SimSampDestination <- SimulationAllSurvGrowth %>%
                                    filter(destination==destination_eval) %>%
                                    sample_n(as.numeric(AllYearsRecruits$n_offs_gen_all_years[i]), replace=F) #sample particles that landed at that destination, number corresponding to the actual number sampled at that site in this year
        
        SimulatedSamplingAll <- bind_rows(SimulatedSamplingAll, SimSampDestination) #build into a sampled particle df
    }

    #assign a numeric ID for each row (which is a sampled particle)
    SimulatedSamplingAll <- SimulatedSamplingAll %>%
        mutate(ParticleSampID=paste ("P", row_number(), sep = "", collapse = NULL))
    
    #nrow(SimulatedSamplingAll)==sum(NGenOffsAll$n_offs_gen) #should be TRUE
    
    #now randomly assign parentage match status to some of these rows
    NumPar <- as.numeric(kernels %>% #should be 37 for 2012-2014
        filter(Year %in% c("2012", "2013", "2014")) %>%
        select(NumParentageMatches) %>%
        summarise(NumParentageMatches=sum(NumParentageMatches)))
    
    SimulatedSamplingAllPar <- SimulatedSamplingAll %>%
        ungroup() %>%
        filter(source %in% NGenOffsAll$site) %>% #we can only assign parentage if we sampled the source site, we've already ensured that we sampled the destination site in the for loop by indexing destination sites to evaluate 
        sample_n(NumPar, replace=F) %>% #sample number of rows consitent with numbers of parentage matches
        mutate(Parentage=1) #%>% #assign these rows a positive match value
        #arrange(date, source, destination) %>%#arrange by the same order as the orginal df, then bind columns together to avoid unexpected repeats from a join
        #select(date, source, destination, Parentage) #drop columns that will result in repeats 
    
    #find the rows that haven't been assigned parentage
    SimulatedSamplingAll_2 <- anti_join((SimulatedSamplingAll %>% ungroup()), SimulatedSamplingAllPar, by=c("ParticleSampID")) 
    
    SimulatedSamplingAll_2$Parentage <- 0 #add in the column for parentage
    
    #combine parentage and non-parentage dfs
    SimulatedSamplingAllPar3 <- bind_rows(SimulatedSamplingAllPar, SimulatedSamplingAll_2) %>%
        mutate(YearSampled=as.character(YearSampled))
    
    
    #NOW FORMAT INTO PARENTAGE MATRIX
    
                PropSampAll <- PropSamp %>%  #add in all sampled sites as possible parent sites
                            filter(end_year =="2014" & PropSamp > 0) %>%
                            #rename(source="site") %>%
                            select(-end_year, -PropSamp)

                #PropSampAll$destination <- PropSampAll$source #make another column for destination
                
                #make a table with the total number of particles sampled at each site
                TotalSimSamp <- SimulatedSamplingAllPar3 %>% 
                            select(source, destination) %>%
                            group_by(destination) %>%
                            summarise(NumSimSampRec=n()) %>%
                            ungroup()
               
                
                #add in all surveyed sites
                TotalSimSamp2 <- left_join(PropSampAll, TotalSimSamp, by=c(site="destination")) %>% #join sites to destination, which will add a row for the surveyed sites with no samples
                    mutate(NumSimSampRec=ifelse(is.na(NumSimSampRec), 0, NumSimSampRec)) %>% #change the Na's that result to 0s, because we didn't get samples there
                    rename(destination="site")#rename sites to destination after joining
                
                
                
                #make a table with the total number of parentage matches found in the simulated loop
                TotalSimPar <- SimulatedSamplingAllPar3 %>% 
                            select(source, destination, Parentage) %>%
                            group_by(source,destination) %>%
                            mutate(NumPar=sum(Parentage)) %>% #sum all the parentage observations along each ROUTE
                            select(-Parentage) %>%
                            ungroup() %>%
                            distinct(source, destination, .keep_all = T)
                
                
                #combine for a summary table BY DESTINATION, this below will be for UNASSIGNED ROWS
                SimSampSummaryAll <- left_join(TotalSimSamp2, TotalSimPar, by="destination") %>%
                    distinct(source, destination, .keep_all = T) %>%
                    group_by(destination) %>%
                    mutate(NumPar=ifelse(is.na(NumPar), 0, NumPar)) %>% #change the Na's that result to 0s, because we didn't get samples there
                    mutate(NumPar=sum(NumPar)) %>% #sum all the parentage observations at each DESTINATION
                    distinct(destination, .keep_all = T) %>%
                    select(-source) %>%
                    mutate(NumUnassigned=NumSimSampRec-NumPar)
                                
                
                #finally, join all of the sites surveyed for a complete parentage matrix
                #for this, need to have a source/destination column for all sites
                PropSampAll <- PropSampAll %>%
                                rename(source="site")
                
                PropSampAll$destination <- PropSampAll$source #make another column for destination
                
                #this below will become parentage matrix! 
                TotalSimPar2 <- full_join(TotalSimPar,PropSampAll, by=c("source", "destination")) %>%
                    mutate(NumPar=ifelse(is.na(NumPar), 0, NumPar))
        
                
                #spread into matrix format
                SimDispMat <- TotalSimPar2 %>%
                    filter(source %in% NGenOffsAll$site) %>%
                    arrange(source, destination)  %>%
                    spread(destination, NumPar) %>%
                    select(-source)
                SimDispMat[is.na(SimDispMat)] <- 0
                
                #add in unassigned row
                unassigned <- t(as.matrix(SimSampSummaryAll %>% arrange(destination) %>% ungroup() %>% select(NumUnassigned)))
                rownames(unassigned)<-NULL
                colnames(unassigned)<-names(SimDispMat)
                
                SimDispMatFullAll <- rbind(SimDispMat, unassigned)
                colnames(SimDispMatFullAll) <- NULL

        #The full remade parentage matrix
        Assignments <- SimDispMatFullAll
    

In [54]:
nrow(Adult_sample_proportions)
Sampled_reefs
#write.table(Adult_sample_proportions, file="~/parentage/kernel_fitting/1340_loci/input/prop_samp_2012-14.csv", row.names=FALSE, quote=FALSE, col.names=FALSE, sep=",")
#write.table(Sampled_reefs, file="~/parentage/kernel_fitting/1340_loci/input/site_index_2012-14.csv", row.names=FALSE, quote=FALSE, col.names=FALSE, sep=",")



[1] 16

1,8,9,14,16,17,18,25,26,27,28,29,30,32,33,34


In [69]:
head(SimulatedKernelsAll)
#write.csv(SimulatedKernelsAll, file="~/oceanography/script_output/KernelFits/1000SimulatedKernelsAllStoppedEarly.csv", row.names=F)

year,k,theta,mdd,med,dist90,iteration
NA,-2.477364,1.1883882,NA,7.10,21.53,1
NA,-2.435259,1.2625946,NA,6.53,19.23,2
NA,-0.315683,0.6688679,NA,1.75,7.51,3
NA,-1.814762,0.7841033,NA,5.84,22.47,4
NA,-2.424130,1.1589185,NA,6.86,21.07,5
NA,-1.766098,0.7476509,NA,6.03,23.92,6


__Seasonal kernel fits__

In [34]:
SimulationSWM <- SimulationAllSurvGrowth %>% #filter by the growth surv adjusted df otherwise
    filter(SimMonsoon=="SWM")
SimulationNEM <- SimulationAllSurvGrowth %>%
    filter(SimMonsoon=="NEM")

In [35]:
#SWM of the corresponding simulation years combined (2011-SWM)

col <- c("season", "k", "theta", "mdd", "med", "dist90", "iteration")
SimulatedKernelsSWM <- as.data.frame(matrix(nrow=0, ncol=7), stringsAsFactors = FALSE)
colnames(SimulatedKernelsSWM) <- col

#don't print warnings for this loop, they are only for setting row names in a tibble which is depracated. But it works. 
options(warn=-1)

pb <- txtProgressBar(min = 0, max = 100, style = 3)

StartTime <- Sys.time()


for(n in 1:100){
        
    #SAMPLE THE SIMULATION DATA, then fit a kernel
    
    SimulatedSamplingSWM = SimulationSWM[FALSE,]
    
    
    for(i in 1:nrow(AllYearsRecruits)){ 
        
        destination_eval <- as.character(AllYearsRecruits$site[i]) #pick out a destination site
        
        SimSampDestination <- SimulationSWM %>%
                                    filter(destination==destination_eval) %>%
                                    sample_n(as.numeric(AllYearsRecruits$n_offs_gen_all_years[i]), replace=F) #sample particles that landed at that destination, number corresponding to the actual number sampled at that site in this year
        
        SimulatedSamplingSWM <- bind_rows(SimulatedSamplingSWM, SimSampDestination) #build into a sampled particle df
    }

    #assign a numeric ID for each row (which is a sampled particle)
    SimulatedSamplingSWM <- SimulatedSamplingSWM %>%
        mutate(ParticleSampID=paste ("P", row_number(), sep = "", collapse = NULL))
    
    #nrow(SimulatedSamplingSWM)==sum(NGenOffsSWM$n_offs_gen) #should be TRUE
    
   #now randomly assign parentage match status to some of these rows
    NumPar <- as.numeric(kernels %>% #should be 37 for 2012-2014
        filter(Year %in% c("2012", "2013", "2014")) %>%
        select(NumParentageMatches) %>%
        summarise(NumParentageMatches=sum(NumParentageMatches)))
    
    SimulatedSamplingSWMPar <- SimulatedSamplingSWM %>%
        ungroup() %>%
        filter(source %in% AllYearsRecruits$site) %>% #we can only assign parentage if we sampled the source site, we've already ensured that we sampled the destination site in the for loop by indexing destination sites to evaluate 
        sample_n(NumPar, replace=F) %>% #sample number of rows consitent with numbers of parentage matches
        mutate(Parentage=1) #%>% #assign these rows a positive match value
        #arrange(date, source, destination) %>%#arrange by the same order as the orginal df, then bind columns together to avoid unexpected repeats from a join
        #select(date, source, destination, Parentage) #drop columns that will result in repeats 
    
    #find the rows that haven't been assigned parentage
    SimulatedSamplingSWM_2 <- anti_join((SimulatedSamplingSWM %>% ungroup()), SimulatedSamplingSWMPar, by=c("ParticleSampID")) 
    
    SimulatedSamplingSWM_2$Parentage <- 0 #add in the column for parentage
    
    #combine parentage and non-parentage dfs
    SimulatedSamplingSWMPar3 <- bind_rows(SimulatedSamplingSWMPar, SimulatedSamplingSWM_2) %>%
        mutate(YearSampled=as.character(YearSampled))
    
    
    #NOW FORMAT INTO PARENTAGE MATRIX
    
                PropSampSWM <- PropSamp %>%  #add in SWM sampled sites as possible parent sites
                            filter(end_year =="2014" & PropSamp > 0) %>%
                            #rename(source="site") %>%
                            select(-end_year, -PropSamp)

                #PropSampSWM$destination <- PropSampSWM$source #make another column for destination
                
                #make a table with the total number of particles sampled at each site
                TotalSimSamp <- SimulatedSamplingSWMPar3 %>% 
                            select(source, destination) %>%
                            group_by(destination) %>%
                            summarise(NumSimSampRec=n()) %>%
                            ungroup()
               
                
                #add in SWM surveyed sites
                TotalSimSamp2 <- left_join(PropSampSWM, TotalSimSamp, by=c(site="destination")) %>% #join sites to destination, which will add a row for the surveyed sites with no samples
                    mutate(NumSimSampRec=ifelse(is.na(NumSimSampRec), 0, NumSimSampRec)) %>% #change the Na's that result to 0s, because we didn't get samples there
                    rename(destination="site")#rename sites to destination after joining
                
                
                
                #make a table with the total number of parentage matches found in the simulated loop
                TotalSimPar <- SimulatedSamplingSWMPar3 %>% 
                            select(source, destination, Parentage) %>%
                            group_by(source,destination) %>%
                            mutate(NumPar=sum(Parentage)) %>% #sum SWM the parentage observations along each ROUTE
                            select(-Parentage) %>%
                            ungroup() %>%
                            distinct(source, destination, .keep_all = T)
                
                
                #combine for a summary table BY DESTINATION, this below will be for UNASSIGNED ROWS
                SimSampSummarySWM <- left_join(TotalSimSamp2, TotalSimPar, by="destination") %>%
                    distinct(source, destination, .keep_all = T) %>%
                    group_by(destination) %>%
                    mutate(NumPar=ifelse(is.na(NumPar), 0, NumPar)) %>% #change the Na's that result to 0s, because we didn't get samples there
                    mutate(NumPar=sum(NumPar)) %>% #sum SWM the parentage observations at each DESTINATION
                    distinct(destination, .keep_all = T) %>%
                    select(-source) %>%
                    mutate(NumUnassigned=NumSimSampRec-NumPar)
                                
                
                #finSWMy, join SWM of the sites surveyed for a complete parentage matrix
                #for this, need to have a source/destination column for SWM sites
                PropSampSWM <- PropSampSWM %>%
                                rename(source="site")
                
                PropSampSWM$destination <- PropSampSWM$source #make another column for destination
                
                #this below will become parentage matrix! 
                TotalSimPar2 <- full_join(TotalSimPar,PropSampSWM, by=c("source", "destination")) %>%
                    mutate(NumPar=ifelse(is.na(NumPar), 0, NumPar))
        
                
                #spread into matrix format
                SimDispMat <- TotalSimPar2 %>%
                    filter(source %in% AllYearsRecruits$site) %>%
                    arrange(source, destination)  %>%
                    spread(destination, NumPar) %>%
                    select(-source)
                SimDispMat[is.na(SimDispMat)] <- 0
                
                #add in unassigned row
                unassigned <- t(as.matrix(SimSampSummarySWM %>% arrange(destination) %>% ungroup() %>% select(NumUnassigned)))
                rownames(unassigned)<-NULL
                colnames(unassigned)<-names(SimDispMat)
                
                SimDispMatFullSWM <- rbind(SimDispMat, unassigned)
                colnames(SimDispMatFullSWM) <- NULL

        #The full remade parentage matrix
        Assignments <- SimDispMatFullSWM
    
    #NOW FIT THE KERNEL


x <- list(Distances=Distances, Assignments=Assignments, Sampled_reefs=Sampled_reefs, Reef_sizes=Reef_sizes, Adult_sample_proportions=Adult_sample_proportions) #put inputs into a list because that's the bbmle format

SimSWMFit <- suppressWarnings(mle2(LL_kt_bbmle, start=list(k=-3, theta=1), lower=c(-10, 0.15), upper=c(10, 8), method="L-BFGS-B", data=x, control=list(maxit=500)))

BestKSWM <- as.numeric(coef(SimSWMFit)[1])
BestThetaSWM <- as.numeric(coef(SimSWMFit)[2])
MDDSWM <- as.numeric(cubintegrate(integrate_kernel_sum1, lower = 0, upper = Inf, k=BestKSWM, theta=BestThetaSWM, , method = "pcubature")$integral)
k_eval <- BestKSWM
theta_eval <- BestThetaSWM
MedSWM  <- round(nleqslv(x = 7, fn = cdf_solve)$x, 2) 
Dist90_SWM <- round(nleqslv(x = 7, fn = cdf_solve90)$x, 2)
    
    #store the info in this df
    SimulatedKernelsSWM_beta <- as.data.frame(matrix(nrow=1, ncol=7), stringsAsFactors = FALSE)
    colnames(SimulatedKernelsSWM_beta) <- col
    
    SimulatedKernelsSWM_beta$season <- NA
    SimulatedKernelsSWM_beta$k <- BestKSWM
    SimulatedKernelsSWM_beta$theta <- BestThetaSWM
    SimulatedKernelsSWM_beta$med <- MedSWM
    SimulatedKernelsSWM_beta$mdd <- MDDSWM
    SimulatedKernelsSWM_beta$dist90 <- Dist90_SWM
    SimulatedKernelsSWM_beta$iteration <- n

    
#join results into larger df
SimulatedKernelsSWM <- bind_rows(SimulatedKernelsSWM, SimulatedKernelsSWM_beta) %>%
    mutate(season="SWM")
    
setTxtProgressBar(pb, n)


}
close(pb)
EndTime <- Sys.time()
EndTime-StartTime
options(warn=0) #turn warnings back on


write.csv(SimulatedKernelsSWM, file="~/oceanography/script_output/KernelFits/100SimulatedKernelsSWMGrowth.csv", row.names=F)

  |======================================================================| 100%


Time difference of 10.43739 mins

In [36]:
#NEM of the corresponding simulation years combined (2011-NEM)

col <- c("season", "k", "theta", "mdd", "med", "dist90", "iteration")
SimulatedKernelsNEM <- as.data.frame(matrix(nrow=0, ncol=7), stringsAsFactors = FALSE)
colnames(SimulatedKernelsNEM) <- col

#don't print warnings for this loop, they are only for setting row names in a tibble which is depracated. But it works. 
options(warn=-1)

pb <- txtProgressBar(min = 0, max = 100, style = 3)

StartTime <- Sys.time()


for(n in 1:100){
        
    #SAMPLE THE SIMULATION DATA, then fit a kernel
    
    SimulatedSamplingNEM = SimulationNEM[FALSE,]
    
    
    for(i in 1:nrow(AllYearsRecruits)){ 
        
        destination_eval <- as.character(AllYearsRecruits$site[i]) #pick out a destination site
        
        SimSampDestination <- SimulationNEM %>%
                                    filter(destination==destination_eval) %>%
                                    sample_n(as.numeric(AllYearsRecruits$n_offs_gen_all_years[i]), replace=F) #sample particles that landed at that destination, number corresponding to the actual number sampled at that site in this year
        
        SimulatedSamplingNEM <- bind_rows(SimulatedSamplingNEM, SimSampDestination) #build into a sampled particle df
    }

    #assign a numeric ID for each row (which is a sampled particle)
    SimulatedSamplingNEM <- SimulatedSamplingNEM %>%
        mutate(ParticleSampID=paste ("P", row_number(), sep = "", collapse = NULL))
    
    #nrow(SimulatedSamplingNEM)==sum(NGenOffsNEM$n_offs_gen) #should be TRUE
    
   #now randomly assign parentage match status to some of these rows
    NumPar <- as.numeric(kernels %>% #should be 37 for 2012-2014
        filter(Year %in% c("2012", "2013", "2014")) %>%
        select(NumParentageMatches) %>%
        summarise(NumParentageMatches=sum(NumParentageMatches)))
    
    SimulatedSamplingNEMPar <- SimulatedSamplingNEM %>%
        ungroup() %>%
        filter(source %in% AllYearsRecruits$site) %>% #we can only assign parentage if we sampled the source site, we've already ensured that we sampled the destination site in the for loop by indexing destination sites to evaluate 
        sample_n(NumPar, replace=F) %>% #sample number of rows consitent with numbers of parentage matches
        mutate(Parentage=1) #%>% #assign these rows a positive match value
        #arrange(date, source, destination) %>%#arrange by the same order as the orginal df, then bind columns together to avoid unexpected repeats from a join
        #select(date, source, destination, Parentage) #drop columns that will result in repeats 
    
    #find the rows that haven't been assigned parentage
    SimulatedSamplingNEM_2 <- anti_join((SimulatedSamplingNEM %>% ungroup()), SimulatedSamplingNEMPar, by=c("ParticleSampID")) 
    
    SimulatedSamplingNEM_2$Parentage <- 0 #add in the column for parentage
    
    #combine parentage and non-parentage dfs
    SimulatedSamplingNEMPar3 <- bind_rows(SimulatedSamplingNEMPar, SimulatedSamplingNEM_2) %>%
        mutate(YearSampled=as.character(YearSampled))
    
    
    #NOW FORMAT INTO PARENTAGE MATRIX
    
                PropSampNEM <- PropSamp %>%  #add in NEM sampled sites as possible parent sites
                            filter(end_year =="2014" & PropSamp > 0) %>%
                            #rename(source="site") %>%
                            select(-end_year, -PropSamp)

                #PropSampNEM$destination <- PropSampNEM$source #make another column for destination
                
                #make a table with the total number of particles sampled at each site
                TotalSimSamp <- SimulatedSamplingNEMPar3 %>% 
                            select(source, destination) %>%
                            group_by(destination) %>%
                            summarise(NumSimSampRec=n()) %>%
                            ungroup()
               
                
                #add in NEM surveyed sites
                TotalSimSamp2 <- left_join(PropSampNEM, TotalSimSamp, by=c(site="destination")) %>% #join sites to destination, which will add a row for the surveyed sites with no samples
                    mutate(NumSimSampRec=ifelse(is.na(NumSimSampRec), 0, NumSimSampRec)) %>% #change the Na's that result to 0s, because we didn't get samples there
                    rename(destination="site")#rename sites to destination after joining
                
                
                
                #make a table with the total number of parentage matches found in the simulated loop
                TotalSimPar <- SimulatedSamplingNEMPar3 %>% 
                            select(source, destination, Parentage) %>%
                            group_by(source,destination) %>%
                            mutate(NumPar=sum(Parentage)) %>% #sum NEM the parentage observations along each ROUTE
                            select(-Parentage) %>%
                            ungroup() %>%
                            distinct(source, destination, .keep_all = T)
                
                
                #combine for a summary table BY DESTINATION, this below will be for UNASSIGNED ROWS
                SimSampSummaryNEM <- left_join(TotalSimSamp2, TotalSimPar, by="destination") %>%
                    distinct(source, destination, .keep_all = T) %>%
                    group_by(destination) %>%
                    mutate(NumPar=ifelse(is.na(NumPar), 0, NumPar)) %>% #change the Na's that result to 0s, because we didn't get samples there
                    mutate(NumPar=sum(NumPar)) %>% #sum NEM the parentage observations at each DESTINATION
                    distinct(destination, .keep_all = T) %>%
                    select(-source) %>%
                    mutate(NumUnassigned=NumSimSampRec-NumPar)
                                
                
                #finNEMy, join NEM of the sites surveyed for a complete parentage matrix
                #for this, need to have a source/destination column for NEM sites
                PropSampNEM <- PropSampNEM %>%
                                rename(source="site")
                
                PropSampNEM$destination <- PropSampNEM$source #make another column for destination
                
                #this below will become parentage matrix! 
                TotalSimPar2 <- full_join(TotalSimPar,PropSampNEM, by=c("source", "destination")) %>%
                    mutate(NumPar=ifelse(is.na(NumPar), 0, NumPar))
        
                
                #spread into matrix format
                SimDispMat <- TotalSimPar2 %>%
                    filter(source %in% AllYearsRecruits$site) %>%
                    arrange(source, destination)  %>%
                    spread(destination, NumPar) %>%
                    select(-source)
                SimDispMat[is.na(SimDispMat)] <- 0
                
                #add in unassigned row
                unassigned <- t(as.matrix(SimSampSummaryNEM %>% arrange(destination) %>% ungroup() %>% select(NumUnassigned)))
                rownames(unassigned)<-NULL
                colnames(unassigned)<-names(SimDispMat)
                
                SimDispMatFullNEM <- rbind(SimDispMat, unassigned)
                colnames(SimDispMatFullNEM) <- NULL

        #The full remade parentage matrix
        Assignments <- SimDispMatFullNEM
    
    #NOW FIT THE KERNEL


x <- list(Distances=Distances, Assignments=Assignments, Sampled_reefs=Sampled_reefs, Reef_sizes=Reef_sizes, Adult_sample_proportions=Adult_sample_proportions) #put inputs into a list because that's the bbmle format

SimNEMFit <- suppressWarnings(mle2(LL_kt_bbmle, start=list(k=-3, theta=1), lower=c(-10, 0.15), upper=c(10, 8), method="L-BFGS-B", data=x, control=list(maxit=500)))

BestKNEM <- as.numeric(coef(SimNEMFit)[1])
BestThetaNEM <- as.numeric(coef(SimNEMFit)[2])
MDDNEM <- as.numeric(cubintegrate(integrate_kernel_sum1, lower = 0, upper = Inf, k=BestKNEM, theta=BestThetaNEM, , method = "pcubature")$integral)
k_eval <- BestKNEM
theta_eval <- BestThetaNEM
MedNEM  <- round(nleqslv(x = 7, fn = cdf_solve)$x, 2) 
Dist90_NEM <- round(nleqslv(x = 7, fn = cdf_solve90)$x, 2)
    
    #store the info in this df
    SimulatedKernelsNEM_beta <- as.data.frame(matrix(nrow=1, ncol=7), stringsAsFactors = FALSE)
    colnames(SimulatedKernelsNEM_beta) <- col
    
    SimulatedKernelsNEM_beta$season <- NA
    SimulatedKernelsNEM_beta$k <- BestKNEM
    SimulatedKernelsNEM_beta$theta <- BestThetaNEM
    SimulatedKernelsNEM_beta$med <- MedNEM
    SimulatedKernelsNEM_beta$mdd <- MDDNEM
    SimulatedKernelsNEM_beta$dist90 <- Dist90_NEM
    SimulatedKernelsNEM_beta$iteration <- n

    
#join results into larger df
SimulatedKernelsNEM <- bind_rows(SimulatedKernelsNEM, SimulatedKernelsNEM_beta) %>%
    mutate(season="NEM")
    
setTxtProgressBar(pb, n)


}
close(pb)
EndTime <- Sys.time()
EndTime-StartTime
options(warn=0) #turn warnings back on


write.csv(SimulatedKernelsNEM, file="~/oceanography/script_output/KernelFits/100SimulatedKernelsNEMGrowth.csv", row.names=F)

  |======================================================================| 100%


Time difference of 41.84831 mins

In [ ]:
#monsoon season prep
NEM_months <- c(11, 12, 1, 2, 3, 4)
SWM_months <- c(5, 6, 7, 8, 9, 10)

NEM <- conn_mat_full4 %>%
    filter(month %in% NEM_months) %>%
    group_by(source, destination) %>%
    summarise(conn=max(fraction))

SWM <- conn_mat_full4 %>%
    filter(month %in% SWM_months) %>%
    group_by(source, destination) %>%
    summarise(conn=max(fraction))

__CELL BELOW IS WORKING FOR GLM OF OBSERVED DATA BY SIMULATED DATA__

In [ ]:
head(GenSimConn)

In [ ]:
sum(is.na(GenSimConn)==T) #should be 0

__run GLM observed genetic dispersal predicted by simulations__

In [ ]:
mean(GenSimConn$obs_disp) #should be 0.024
var(GenSimConn$obs_disp) #should be 0.054
#overdispersion?
#no, because residual deviance of the model is not greater than the df
#Residual deviance:  38.221  on 948  degrees of freedom

In [ ]:
#plan:
#fit 3 successive log-linear models starting with main efects only(fit1), main effects plus all 2-way interactions (fit2), and added in all 3 way interactions (fit3)


In [ ]:
NullMod <- glm(obs_disp ~ dist_km , data=GenSimConn, family="poisson")
summary(NullMod)

In [ ]:
ObsModExpYearFit1 <- glm(obs_disp ~ AnnRecPart + YearSampled + direction + dist_km +SourcePropSamp +DestPropSamp, data=GenSimConn, family="poisson")


In [ ]:
summary(ObsModExpYearFit1)

In [ ]:
#only main effects, first find worthwhile predictors with AIC
#the intercept is very significant but I think that just means 0s are more common in the data. totally true!

ObsModExpYearFit1 <- glm(obs_disp ~ AnnRecPart + YearSampled + direction + dist_km +SourcePropSamp +DestPropSamp, data=GenSimConn, family="poisson")
MainEff_AnnRecPart <- update(ObsModExpYearFit1, . ~ . -AnnRecPart)
MainEff_YearSampled <- update(ObsModExpYearFit1, . ~ . -YearSampled)
MainEff_direction <- update(ObsModExpYearFit1, . ~ . -direction)
MainEff_dist_km <- update(ObsModExpYearFit1, . ~ . -dist_km)
MainEff_SourcePropSamp <- update(ObsModExpYearFit1, . ~ . -SourcePropSamp)
MainEff_DestPropSamp <- update(ObsModExpYearFit1, . ~ . -DestPropSamp)
#drop the 3 terms AIC doesn't support? maybe not though, because when there's an interaction that's the best model
MainEff_ARP_DPrp_Dist <- glm(obs_disp ~ YearSampled + direction +SourcePropSamp, data=GenSimConn, family="poisson")
#add the interaction between year and simulations
ObsModExpYearFit2IntYear <- glm(obs_disp ~ AnnRecPart*YearSampled + direction+SourcePropSamp, data=GenSimConn, family="poisson")
ObsModExpYearFit2IntDir <- glm(obs_disp ~ AnnRecPart*YearSampled + direction+SourcePropSamp, data=GenSimConn, family="poisson")
ObsModExpYearFit2IntDirNoROMS <- glm(obs_disp ~ AnnRecPart+YearSampled*direction+SourcePropSamp, data=GenSimConn, family="poisson")
ObsModExpYearFit3Int <- glm(obs_disp ~ AnnRecPart*YearSampled*direction+SourcePropSamp, data=GenSimConn, family="poisson")

FullModNoROMSDestPropSamp <- glm(obs_disp ~  YearSampled + direction + dist_km +SourcePropSamp , data=GenSimConn, family="poisson")
FullModNoROMSDestPropSampInt <- glm(obs_disp ~  YearSampled* direction + dist_km +SourcePropSamp , data=GenSimConn, family="poisson")

MainEffAIC <- as.data.frame(AIC(NullMod, FullModNoROMSDestPropSampInt, FullModNoROMSDestPropSamp, ObsModExpYearFit1, MainEff_AnnRecPart, MainEff_YearSampled, MainEff_direction, MainEff_dist_km, MainEff_SourcePropSamp, MainEff_DestPropSamp, ObsModExpYearFit2IntDir, ObsModExpYearFit2IntYear, ObsModExpYearFit2IntDirNoROMS, ObsModExpYearFit3Int))
MainEffAIC$model <- row.names(MainEffAIC)
MainEffAIC <- MainEffAIC %>% arrange(AIC)
MainEffAIC
#/write.csv(MainEffAIC, file="~/oceanography/script_output/GLM/GLMObsDispAIC.csv",row.names=F,  quote=F)


In [ ]:
FullModNoROMSDestPropSampInt <- glm(obs_disp ~  YearSampled* direction+direction+YearSampled + dist_km +SourcePropSamp , data=GenSimConn, family="poisson")

summary(FullModNoROMSDestPropSampInt)

In [ ]:
AIC(test, MainEff_AnnRecPart,MainEff_DestPropSamp )

In [ ]:
summary(ObsModExpYearFit2Int ) #judging by the significant intercept (which should be AnnRecPart:Year2012?) when there's more particles expected from the ROMS model, there's actually less in the observed data.
ObsModExpYearFit2Int_Sum <- tidy(ObsModExpYearFit2Int)
#write.csv(ObsModExpYearFit2Int_Sum, file="~/oceanography/script_output/GLM/YearDispIntBestModSum.csv",row.names=F,  quote=F)


In [ ]:
ROMSFull <- glm(AnnRecPart ~ as.factor(SimYear) + direction + dist_km + source + destination, data=GenSimConn, family="poisson")
#MainEff_AnnRecPart <- update(ObsModExpYearFit1, . ~ . -AnnRecPart)

In [ ]:
summary(ROMSFull)

In [ ]:
summary(SimConnDFDir$NormSourceMonsoon)

In [ ]:
#what are the routes that create these peaks in direction annually?

#are there similar patterns seasonally?

#plot the connectivity matrix with of the roms model

In [ ]:
head(SimConnDFMeta %>% dplyr::select(AnnRecPart, direction, SimYear, source, destination))
SimConnDFMeta %>%

In [ ]:
hist(SimConnDFMeta$direction, breaks=100)

In [ ]:
summary(SimConnDFMeta$direction)
(SimConnDFMeta$direction)

In [ ]:
#this isn't significant, which means the residual difference is small enough here to say the model fits reasonably well
with(test_mod, cbind(res.deviance = deviance, df = df.residual,
  p = pchisq(deviance, df.residual, lower.tail=FALSE)))


In [ ]:
#how much devience is explained here?

In [ ]:
plot(obs_disp ~ ExpDisp, data=SimPlusGenWithDist)

In [ ]:
#plot(test_mod_int)

In [ ]:
#plot predicted model
range(SimPlusGenWithDist$ExpDisp)

In [ ]:
exp_disp_range <- seq(0, 66114, 1)

In [ ]:
obs_disp_pred <- predict(test_mod_int,type="response")

In [ ]:
plot( SimPlusGenWithDist$ExpDisp, SimPlusGenWithDist$obs_disp, pch = 16)#, xlab = "WEIGHT (g)", ylab = "VS")

lines(obs_disp_pred)